# 分布鲁棒优化

**作者：**  
Daniel Kuhn  
风险分析与优化主席，  
洛桑联邦理工大学，瑞士洛桑  
电子邮件：[daniel.kuhn@epfl.ch](mailto:daniel.kuhn@epfl.ch)

Soroosh Shafiee  
康奈尔大学，运营研究与信息工程学院，  
美国纽约伊萨卡  
电子邮件：[shafiee@cornell.edu](mailto:shafiee@cornell.edu)

Wolfram Wiesemann  
帝国理工学院商学院，  
英国伦敦帝国理工大学  
电子邮件：[ww@imperial.ac.uk](mailto:ww@imperial.ac.uk)

---

## 摘要

分布鲁棒优化（DRO）研究不确定条件下的决策问题，其中控制不确定参数的概率分布也不确定。DRO模型的一个关键组成部分是其模糊集，即满足已知结构信息或统计信息的概率分布族。DRO寻找在模糊集中的最坏分布下表现最好的决策。这一最坏情况(悲观)标准得到了心理学和神经科学研究的支持，研究表明许多决策者对分布不确定性有较低的容忍度。DRO根植于统计学、运筹学和控制理论，近期研究揭示了其与机器学习中的正则化技术和对抗性训练之间的深刻联系。本综述以统一且独立的方式呈现了该领域的关键发现。

---

## 目录

1. [引言](#引言)
2. [模糊集](#模糊集)
3. [模糊集的拓扑属性](#模糊集的拓扑属性)
4. [最坏期望问题的对偶理论](#最坏期望问题的对偶理论)
5. [最坏风险问题的对偶理论](#最坏风险问题的对偶理论)
6. [自然子问题的解析解](#自然子问题的解析解)
7. [自然子问题的有限凸重构](#自然子问题的有限凸重构)
8. [通过鲁棒化进行正则化](#通过鲁棒化进行正则化)
9. [DRO问题的数值求解方法](#dro问题的数值求解方法)
10. [统计保证](#统计保证)
11. [参考文献](#参考文献)


# 1. 引言 (Introduction)
数学优化研究以下形式的问题：

$$
\inf _{x \in \mathcal{X}} \ell(x),
$$

其中，决策变量 $x$ 从可行解组成的集合 $\mathcal{X} \subseteq \mathbb{R}^n$ 中选择，目的是最小化损失函数 $\ell: \mathbb{R}^n \rightarrow \overline{\mathbb{R}}$。数学优化起源于17世纪末艾萨克·牛顿、戈特弗里德·威廉·莱布尼茨、皮埃尔·德·费马等人对微积分的研究，至今已有丰富的历史，涉及许多数学家、经济学家、工程师和科学家的贡献。现代数学优化的诞生通常归功于乔治·丹茨格，他在1947年提出的单纯形法可以求解线性优化问题，其中损失函数 $\ell$ 是仿射的，且可行集 $\mathcal{X}$ 是一个多面体（Dantzig 1956）。随后的重要进展包括：凸分析的丰富理论（Rockafellar 1970），以及线性优化问题的多项式时间求解方法（Khachiyan 1979，Karmarkar 1984）以及许多类别的非线性凸优化问题（Nesterov 和 Nemirovskii 1994）。

经典的优化问题是确定性的，即假设所有问题数据都已知且确定。然而，实践中遇到的大多数决策问题都依赖于那些受到测量误差影响的参数，或者仅在决策必须做出并且实施之后才能揭示的参数。将受到不确定性影响的决策问题简单建模为确定性优化问题的一种方法是用期望值或适当的点预测值替代所有不确定的参数。然而，早在很久以前就有人发现并记录了这样一个事实：那些将不确定的参数替换为其均值的决策者往往会陷入“平均误差”的陷阱（Savage, Scholtes 和 Zweidler 2006，Savage 2012）。为了考虑那些偏离均值的不确定性实现，Beale（1955）和Dantzig（1955）独立提出了以下形式的随机规划：

$$
\inf _{x \in \mathcal{X}} \mathbb{E}_{\mathbb{P}}[\ell(x, Z)],
$$

这些方法明确地将不确定的问题参数 $Z$ 建模为一个服从概率分布 $\mathbb{P}$ 的随机向量，并且在此基础上寻找在期望下表现最好的决策（或根据某些风险度量进行优化）。此后，随机规划已发展成为一个成熟的领域（Birge 和 Louveaux 2011，Shapiro, Dentcheva 和 Ruszczyński 2009），并为机器学习中的经验风险最小化原则提供了理论基础（Bishop 2006，Hastie, Tibshirani 和 Friedman 2009）。

尽管随机规划在理论和实践中取得了成功，但它至少存在两个缺点。首先，其假设概率分布 $\mathbb{P}$ 是完全已知的，在许多实际情况下是不现实的， 且随机规划对分布$\mathbb{P}$ 的误设非常敏感。不同领域对这一效应的描述有：优化者的诅咒（Smith 和 Winkler 2006），优化的误差最大化效应（Michaud 1989，DeMiguel 和 Nogales 2009），优化偏差（Shapiro 2003）或过拟合（Bishop 2006，Hastie 等 2009）。其次，评估固定决策的期望损失需要计算多维积分，而这对于非常简单的损失函数和分布来说也非常复杂。因此，随机规划受到维度灾难的挑战，即其计算复杂度通常随着随机向量 $Z$ 的维度呈指数增长。为了缓解这两个缺点，Soyster（1973）提出将不确定性影响的决策问题建模为鲁棒优化问题，形式为：

$$
\inf _{x \in \mathcal{X}} \sup _{z \in \mathcal{Z}} \ell(x, z).
$$

鲁棒优化用基于集合的描述代替了不确定问题参数的概率描述，寻找在参数$z$最坏情况下的实现上表现最好的决策。经过长时间的忽视，Soyster（1973）的思想在90年代末得到了重新审视并大大扩展，Kouvelis 和 Yu（1997），El Ghaoui、Oustry 和 Lebret（1998），El Ghaoui 和 Lebret（1998a,b），BenTal 和 Nemirovski（1999b, 1998, 1999a），Bertsimas 和 Sim（2004）等人进一步推进了该领域。有关鲁棒优化文献的综述，请参见 Ben-Tal、El Ghaoui 和 Nemirovski（2009），Rustem 和 Howe（2009）和 Bertsimas 和 den Hertog（2022）。我们指出，类似的思想在鲁棒稳定性（Horn 和 Johnson 1985，Doyle、Glover、Khargonekar 和 Francis 1989，Green 和 Limebeer 1995）和鲁棒控制（Zames 1966，Khalil 1996，Zhou、Doyle 和 Glover 1996）领域中也得到了独立发展，这些领域研究系统在参数变化下是否能够保持稳定，或者在参数变化下设计出具有良好性能的系统。有关鲁棒稳定性和控制的教材介绍，请参见 Zhou 和 Doyle（1999）和 Dullerud 和 Paganini（2001）。Hansen 和 Sargent（2008）将鲁棒控制技术应用于受模型不确定性影响的经济问题，设计在多种可能的模型误设下都能表现良好的政策。

尽管鲁棒优化减少了随机规划中的信息和计算负担，但它对模糊集内所有参数实现的平等对待且聚焦于最坏情境，可能导致其在实际应用中过于保守。这些问题促使研究人员研究分布鲁棒优化问题，形式为：

$$
\inf _{x \in \mathcal{X}} \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(x, Z)],
$$

该模型将不确定的参数$Z$建模为一个随机向量，其由一个模糊集$\mathcal{P}$中的某个分布$\mathbb{P}$支配，寻求在最坏分布$\mathbb{P} \in \mathcal{P}$下，其期望值表现最好的决策。分布鲁棒优化（DRO）因此将随机规划的分布视角与鲁棒优化的最坏情况聚焦结合在一起。Herbert E. Scarf通常被认为是这一方法的先驱，他在关于新闻供应商问题的研究中提出，未知的需求分布仅通过其均值和方差来表征（Scarf 1958）。随后，Dupačová（1966，1987，1994）和Shapiro与Kleywegt（2002）研究了那些模糊集规定了未知概率分布的支持集、某些低阶矩、独立模式或其他结构特性的问题。Ermoliev、Gaivoronski 和 Nedeva（1985）及 Gaivoronski（1991）为基于矩模糊集的DRO问题开发了早期的求解方法。现代DRO的出现通常归功于Bertsimas和Popescu（2002，2005）的工作，他们在部分分布信息下推导了概率不等式，并将其技术应用于期权定价问题；El Ghaoui、Oks和Oustry（2003）以及Calafiore和El Ghaoui（2006）的研究，他们研究了DRO问题最小化目标函数的某个分位数，或一组受不确定性影响的约束应当以高概率满足，跨所有具有已知矩约束的概率分布；以及Delage和Ye（2010）的研究，其研究了类似的DRO问题，目标为最小化最坏情况下的期望目标值。

早期关于DRO的研究主要集中在矩模糊集上，这些模糊集包含所有满足有限多个矩约束的指定支持集$\mathcal{Z}$上的分布。与随机规划不同，具有矩模糊集的DRO问题有时会在随机向量$Z$的维度上展现出有利的扩展性。然而，截然不同的分布可能具有相同的矩。因此，矩模糊集总是包含广泛的分布，其中包括一些不太可能的分布，当有充足的历史数据时，这些不太可能的分布可以安全地被排除。这促使Ben-Tal、den Hertog、De Waegenaere、Melenberg和Rennen（2013）以及Wang、Glynn和Ye（2016）引入了模糊集，这些模糊集包含所有在某个参考分布的邻域内的分布（通常是从历史数据形成的经验分布）。这些邻域可以通过概率分布之间的距离函数来定义，例如$\phi$-散度（Csiszar 1963）或Wasserstein距离（Villani 2008）。与矩模糊集不同，基于统计距离的模糊集具有可调的大小参数（例如，半径），因此可以缩小为仅包含参考分布的单例。如果参考分布收敛到未知的真实分布，并且随着历史数据的增加模糊集大小参数趋近于0，则DRO问题最终会简化为在真实分布下的经典随机规划。早期关于基于差异的模糊集的研究假定$Z$是一个离散随机向量，具有有限的支持集$\mathcal{Z}$。基于差异的DRO问题扩展到一般的（可能是连续）随机向量的工作由Mohajerin Esfahani 和 Kuhn（2018），Zhao 和 Guan（2018），Blanchet 和 Murthy（2019），Zhang、Yang 和 Gao（2024b）以及 Gao 和 Kleywegt（2023）完成，他们使用最优传输差异构建了模糊集。我们请参阅Kuhn、Mohajerin Esfahani、Nguyen 和 Shafieezadeh-Abadeh（2019）以及Rahimian 和 Mehrotra（2022）以获取DRO文献的先前综述。

“分布鲁棒性”一词最早源于鲁棒统计。Huber（1981）首次提出该术语，用以描述在数据中存在异常值时做出鲁棒决策的方法。这个概念扩展了Box（1953，1979）的早期工作，其研究了在基础分布偏离正态分布时的鲁棒性，正态分布是许多统计模型中的常见假设。为了应对异常值带来的挑战，统计学家们发展了几种污染模型，每个模型提供了不同的解决数据异常的途径。Huber污染模型由Huber（1964，1968）提出，并由Hampel（1968，1971）进一步发展，假设观察到的数据是从真实分布和任意污染分布的混合中抽取的。邻域污染模型通过统计距离定义偏离真实分布的情况，例如总变差（Donoho 和 Liu 1988）或Wasserstein距离（Zhu、Jiao 和 Steinhardt 2022a，Liu 和 Loh 2023）。最近，数据依赖的自适应污染模型允许将部分观察到的数据点替换为从任意分布中抽取的点（Diakonikolas、Kamath、Kane、Li、Moitra 和 Stewart 2019，Zhu 等 2022a）。有趣的是，DRO模型的乐观对偶模型，优化的是模糊集中的最佳分布（而非最坏分布），恢复了许多来自鲁棒统计的估计量（Blanchet、Li、Lin 和 Zhang 2024b，Jiang 和 Xie 2024）。关于算法鲁棒统计的最新进展，请参阅Diakonikolas 和 Kane（2023）。

鲁棒优化和分布鲁棒优化在机器学习中找到了广泛的应用。例如，机器学习文献中的流行正则化器具有鲁棒性解释，这为正则化在实践中的强大表现提供了理论洞察（Xu，Caramanis 和 Mannor 2009，Shafieezadeh-Abadeh、Kuhn 和 Mohajerin Esfahani 2019，Li、Lin、Blanchet 和 Nguyen 2022，Gao、Chen 和 Kleywegt 2024）。同样，DRO模型的乐观对偶模型，优化的是模糊集中的最佳（而非最坏）分布，产生了上置信区间算法，这些算法在赌博和强化学习文献中非常普遍（Blanchet 等 2024b，Jiang 和 Xie 2024）。DRO也与对抗训练有关，对抗训练旨在通过在对抗性样本的视角下训练模型，从而提高机器学习模型的泛化性能（Goodfellow、Shlens 和 Szegedy 2015）。对抗样本是现有数据点的扰动，旨在误导模型做出不正确的预测。


DRO 与扩展的随机（动态）规划之间也存在深刻的联系，这些扩展通过用一致的风险度量代替期望值。与期望值相似，风险度量将随机变量映射到扩展实数上。与期望值不同，期望值是风险中立的，因为它对正负结果赋予相同的权重，而风险度量通常赋予负面结果更大的权重，从而考虑到决策者经常表现出的风险厌恶。Artzner、Delbaen、Eber 和 Heath (1999) 以及 Delbaen (2002) 表明，满足一致性公理以及 Fatou 性质的风险度量可以等效地表示为在特定分布集上的最坏情况期望值。换句话说，优化最坏情况期望值（如DRO中所做的）和优化一致风险度量之间有直接的联系。在金融数学文献中，针对一类非线性期望——所谓的 $G$-期望，已经发展了类似的表示定理，这些期望基于反向随机微分方程的解（Peng 1997, 2007a,b, 2019）。Peng (2023) 显示，次线性 $G$-期望等价于在特定分布族上的最坏情况期望值，从而在 $G$-期望理论和DRO之间架起了桥梁。

从哲学上讲，DRO 与模糊厌恶原则相关，即个体更喜欢已知风险而不是未知风险，即使未知风险可能带来更高的回报。在经济学文献中，风险性结果的概率已知与模糊性结果的概率（部分）未知之间的区别可以追溯到至少凯恩斯（1921）和奈特（1921）。模糊厌恶的概念通过Ellsberg悖论（Ellsberg 1961）广泛传播，这是一种思想实验，在该实验中，人们被要求在一个已知分布的彩色球（例如50个红球和50个蓝球）的盒子和一个具有未知分布的相同彩色球的盒之间做出选择（即红蓝球的比例未知）。尽管可能有相等或更好的赔率，许多人仍然更愿意在已知分布的盒上下注，即他们表现出模糊厌恶。Ellsberg悖论挑战了经典的期望效用理论，它导致了如最大最小期望效用理论（Gilboa 和 Schmeidler 1989）等扩展，作为DRO的理论基础。之后，模糊厌恶在金融市场（Epstein 和 Miao 2003，Bossaerts、Ghirardato、Guarnaschelli 和 Zame 2010）、保险市场（Cabantous 2007）、个人决策（Dimmock、Kouwenberg 和 Wakker 2016）、宏观经济政策（Hansen 和 Sargent 2010）、拍卖（Salo 和 Weber 1995）以及信任博弈（Li、Turmunkh 和 Wakker 2019b）中被发现。

同时，也有大量医学和神经科学证据支持模糊厌恶的存在。Hsu、Bhatt、Adolphs、Tranel 和 Camerer（2005）发现，当个体面临模糊性时，大脑中的关键情绪处理中心——杏仁体的活动比面对已知概率的情境时更为活跃，这表明杏仁体在驱动模糊厌恶中的作用。Krain、Wilson、Arbuckle、Castellanos 和 Milham（2006）的荟萃分析突出了前额叶皮层的作用，该区域负责高级认知控制、理性决策和情绪调节，用于处理模糊性。此外，Wu、Sun、Camilleri、Eickhoff 和 Yu（2021）进行的荟萃分析表明，风险和模糊的处理都依赖于前岛叶。风险处理还会激活背内侧前额叶皮层和腹侧纹状体，而模糊处理则特定地涉及背外侧前额叶皮层、下顶叶和右前岛叶。这支持了这样一个观点：当个体面对模糊决策与风险决策时，涉及的是不同的神经机制。遗传因素可能影响个体对模糊厌恶的倾向。He、Xue、Chen、Lu、Dong、Lei、Ding、Li、Li、Chen、Li、Moyzis 和 Bechara（2010）将某些遗传多态性与个体在风险和模糊决策下的表现联系起来。在另一项研究中，Buckert、Schwieren、Kudielka 和 Fiebach（2014）研究了荷尔蒙变化（例如，与压力和焦虑相关的皮质醇水平升高）如何影响风险和模糊决策。这些发现共同表明，风险和模糊的感知不仅仅是认知现象，还受到大脑结构、遗传和荷尔蒙因素的影响，这些因素塑造了个体在模糊性决策中的差异。最后，我们提到Hartley 和 Somerville（2015）以及 Blankenstein、Crone、van den Bos 和 van Duijvenvoorde（2016），他们研究了模糊厌恶如何在儿童、青少年和成人之间有所不同，Hayden、Heilbronner 和 Platt（2010）观察到，当恒河猕猴在选择风险和模糊游戏时，表现出模糊厌恶。

我们的分析的一个重要部分是研究最坏期望值 $\sup _{\mathrm{P} \in \mathcal{P}} \mathbb{E}_{\mathrm{P}}[\ell(x, Z)]$，这是DRO问题（1.2）的目标函数。评估这个表达式通常需要求解一个半无穷优化问题，这个问题有无限多个变量来表征概率分布 $P$，并且受到模糊集 $\mathcal{P}$ 施加的有限个约束。我们称这个问题为“自然的子问题”，它是区分DRO问题（1.2）与确定性、随机和鲁棒优化问题的关键特征。本综述的其余部分结构如下：第2和第3节回顾了常见的模糊集 $\mathcal{P}$ 及其拓扑性质，特别关注自然子问题在什么条件下能够达到最优值。第4和第5节为自然子问题开发了对偶性理论，使我们能够等价地重新表述最坏期望值或其上界为一个半无限优化问题：其具有有限多个对偶决策变量，无限多个约束。这个对偶性框架为第6节自然子问题的解析解奠定了基础，其依赖于构建原始和对偶可行解，二者具有强对偶性从而产生相同的目标值。第7和第8节利用相同的对偶性理论，开发了自然子问题以及整个DRO问题（1.2）的等价重构和保守逼近。第9节展示了对偶性理论如何为自然子问题和整个DRO问题提供数值求解技术。最后，第10节回顾了不同模糊集所享有的统计保障。

由于篇幅限制，本综述在选择涵盖的主题时面临艰难的取舍。我们决定聚焦于最常用的模糊集，并简要回顾其他可能的选择，例如边际模糊集、具有结构约束的模糊集（包括对称性和单峰性）、Sinkhorn模糊集或条件相对熵模糊集。同样，我们没有涵盖分布优先优化和决策随机化等重要但略显复杂的话题。最后，我们专注于单阶段问题，其中不确定性在作出此时此刻的决策 $x \in \mathcal{X}$ 后完全解决；至于不确定性随时间实现并且可以进行响应决策的两阶段和多阶段DRO问题，已由 Delage 和 Iancu（2015）以及 Yanikoglu、Gorissen 和 den Hertog（2019）进行了综述。





## 1.1 符号

本文中考虑的所有向量空间都定义在实数域上。为了简洁起见，我们简单地称它们为“向量空间”而不是“实向量空间”。我们使用 $\overline{\mathbb{R}} = \mathbb{R} \cup \{-\infty, \infty\}$ 来表示扩展实数。函数 $f: \mathbb{R}^d \rightarrow \overline{\mathbb{R}}$ 的有效域定义为 $\operatorname{dom}(f) = \{z \in \mathbb{R}^d : f(z) < \infty\}$，而 $f$ 的上图定义为 $\operatorname{epi}(f) = \{(z, \alpha) \in \mathbb{R}^d \times \mathbb{R}: f(z) \leq \alpha\}$。当且仅当 $\operatorname{dom}(f) \neq \emptyset$ 且对于所有 $z \in \mathbb{R}^d$，$f(z) > -\infty$时，我们称 $f$ 为适当的。$f$ 的凸共轭是函数 $f^*: \mathbb{R}^d \rightarrow \overline{\mathbb{R}}$，通过 $f^*(y) = \sup_{z \in \mathbb{R}^d} y^{\top} z - f(z)$ 定义。一个凸函数 $f$ 被称为闭合的，如果它是适当的并且下半连续，或者它恒等于 $+\infty$ 或 $-\infty$。可以证明，$f$ 是闭合的当且仅当它与其双共轭 $f^{**}$ 一致，即与 $f^*$ 的共轭一致。如果 $f$ 是适当的、凸的且下半连续的，那么它的退化函数 $f^{\infty}: \mathbb{R}^d \rightarrow \mathbb{R}$ 通过以下公式定义：$f^{\infty}(z) = \lim_{\alpha \to \infty} \alpha^{-1}(f(z_0 + \alpha z) - f(z_0))$，其中 $z_0$ 是 $\operatorname{dom}(f)$ 中的任意一点（Rockafellar 1970，第8.5定理）。$f$ 的透视图是函数 $f^\pi: \mathbb{R}^d \times \mathbb{R} \rightarrow \overline{\mathbb{R}}$，通过以下公式定义：$f^\pi(z, t) = t f(z / t)$（当 $t > 0$ 时），$f^\pi(z, t) = f^{\infty}(z)$（当 $t = 0$ 时），$f^\pi(z, t) = \infty$（当 $t < 0$ 时）。可以证明，$f^\pi$ 是适当的、凸的且下半连续的（Rockafellar 1970，第67页）。在没有混淆的风险时，我们偶尔使用 $t f(z / t)$ 来表示 $f^\pi(z, t)$，即使 $t = 0$。集合 $\mathcal{Z} \subseteq \mathbb{R}^d$ 的指示函数 $\delta_{\mathcal{Z}}: \mathbb{R}^d \rightarrow \mathbb{R}$ 定义为：如果 $z \in \mathcal{Z}$，则 $\delta_{\mathcal{Z}}(z) = 0$；如果 $z \notin \mathcal{Z}$，则 $\delta_{\mathcal{Z}}(z) = \infty$。$\delta_{\mathcal{Z}}$ 的共轭 $\delta_{\mathcal{Z}}^*$ 被称为 $\mathcal{Z}$ 的支持函数。它满足 $\delta_{\mathcal{Z}}^*(y) = \sup_{z \in \mathcal{Z}} y^{\top} z$。随机对象用大写字母表示（例如，$Z$），其实现用小写字母表示（例如，$z$）。对于任意闭集 $\mathcal{Z} \subseteq \mathbb{R}^d$，我们使用 $\mathcal{M}(\mathcal{Z})$ 来表示所有有限签名的Borel测度空间，而 $\mathcal{M}_+(\mathcal{Z})$ 表示 $\mathcal{M}(\mathcal{Z})$ 中所有（非负的）Borel测度的凸锥，$\mathcal{P}(\mathcal{Z})$ 表示 $\mathcal{M}_+(\mathcal{Z})$ 中所有概率分布的凸集。关于 $P \in \mathcal{P}(\mathcal{Z})$ 的期望算子定义为：$\mathbb{E}_{P}[f(Z)] = \int_{\mathcal{Z}} f(z) \mathbb{dP}(z)$，对于任何 Borel 函数 $f: \mathcal{Z} \rightarrow \overline{\mathbb{R}}$。如果 $f$ 的正负部分的积分都为 $\infty$，则我们“敌对”地定义 $\mathbb{E}_{P}[f(Z)]$。即，如果积分出现在最小化（最大化）问题的目标函数中，则我们设定 $\mathbb{E}_{P}[f(Z)] = \infty$（或 $-\infty$）。Dirac 概率分布将单位概率赋给 $z \in \mathcal{Z}$，记作 $\delta_z$。Dirac 分布 $\delta_z$ 不应与单点集合 $\{z\}$ 的指示函数 $\delta_{\{z\}}$ 混淆。对于任何 $\mathcal{P} \in \mathcal{P}(\mathcal{Z})$ 和任何 Borel 可测变换 $f: \mathcal{Z} \rightarrow \mathcal{Z}^{\prime}$，其中 $\mathcal{Z} \subseteq \mathbb{R}^d$ 和 $\mathcal{Z}^{\prime} \subseteq \mathbb{R}^{d^{\prime}}$，我们用 $\mathbb{P} \circ f^{-1}$ 表示在 $f$ 下的推前分布。因此，如果 $Z$ 是一个由 $P$ 支配的在 $\mathcal{Z}$ 上的随机向量，那么 $f(Z)$ 就是一个由 $\mathbb{P} \circ f^{-1}$ 支配的在 $\mathcal{Z}^{\prime}$ 上的随机向量。一个集合 $\mathcal{Z} \subseteq \mathbb{R}^d$ 的闭包、内点和相对内点分别用 $\mathrm{cl}(\mathcal{Z})$、$\operatorname{int}(\mathcal{Z})$ 和 $\operatorname{rint}(\mathcal{Z})$ 表示。我们使用 $\mathbb{R}_+^d$ 和 $\mathbb{R}_{++}^d$ 来表示 $\mathbb{R}^d$ 中的非负正交体及其内点。此外，我们使用 $\mathbb{S}^d$ 来表示所有对称矩阵的空间，$\mathbb{R}^{d \times d}$ 中的对称矩阵。正半定矩阵的锥体用 $\mathbb{S}_+^d$ 表示，而 $\mathbb{S}_{++}^d$ 表示其内点，即所有正定矩阵的集合。逻辑命题 $\mathbb{1}_{\mathcal{E}}$ 的真值为1，当且仅当 $\mathcal{E}$ 为真，否则为0。所有自然数集合 $\{1, 2, 3, \ldots\}$ 用 $\mathbb{N}$ 表示，$[n] = \{1, \ldots, n\}$ 表示所有小于等于 $n$ 的整数的集合。

# 2. 模糊集合

模糊集合 $\mathcal{P}$ 是一个概率分布族，定义在一个公共的可测空间上。本文中，我们假设 $\mathcal{P} \subseteq \mathcal{P}(\mathcal{Z})$，其中 $\mathcal{P}(\mathcal{Z})$ 表示所有 Borel 概率分布的全集，这些分布定义在闭集 $\mathcal{Z} \subseteq \mathbb{R}^d$ 上。本节回顾了几种常见的模糊集合类别。对于每个类别，我们首先给出正式定义并提供历史背景信息。随后，我们举出重要的模糊集合实例，并突出它们的应用。




### 2.1. 矩量模糊集合

矩量模糊集合是一个满足有限个（广义的）矩量条件的概率分布族。形式上，它可以表示为：

$$
\mathcal{P}=\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{E}_{\mathrm{P}}[f(\mathrm{Z})] \in \mathcal{F}\right\},
$$

其中 $f: \mathcal{Z} \rightarrow \mathbb{R}^m$ 是一个 Borel 可测矩量函数，$\mathcal{F} \subseteq \mathbb{R}^m$ 是一个不确定性集合。根据定义，矩量模糊集合（2.1）包含所有支持在 $\mathcal{Z}$ 上的概率分布 $P$，其广义矩量 $\mathbb{E}_{\mathrm{P}}[f(Z)]$ 是定义良好的，并且属于不确定性集合 $\mathcal{F}$。类型为（2.1）的模糊集合首次由 Isii $(1960, 1962)$ 和 Karlin 与 Studden (1966) 研究，用于建立广义切比雪夫不等式的精确度。以下小节回顾了矩量模糊集合的常见实例。


#### 2.1.1. 仅支持模糊集合

仅支持模糊集合包含所有支持在 $\mathcal{Z} \subseteq \mathbb{R}^d$ 上的概率分布，即 $\mathcal{P} = \mathcal{P}(\mathcal{Z})$。它可以视为（2.1）的一种实例，设 $f(z) = 1$ 且 $\mathcal{F} = \{1\}$。任何具有模糊集合 $\mathcal{P}(\mathcal{Z})$ 的 DRO 问题，本质上都等同于一个经典的鲁棒优化问题，其不确定性集合为 $\mathcal{Z}$，即：

$$
\inf _{x \in \mathcal{X}} \sup _{P \in \mathcal{P}(\mathcal{Z})} \mathrm{B}_{\mathrm{P}}[\ell(x, Z)] = \inf _{x \in \mathcal{X}} \sup _{z \in \mathcal{Z}} \ell(x, z) .
$$

有关鲁棒优化的理论和应用的综合回顾，请参见 (Ben-Tal 和 Nemirovski 1998, 1999a, 2000, 2002, Bertsimas 和 Sim 2004, Ben-Tal 等人 2009, Bertsimas, Brown 和 Caramanis 2011, Ben-Tal, den Hertog 和 Vial 2015a, Bertsimas 和 den Hertog 2022)。

如果不确定集合 $\mathcal{Z}$ 覆盖了某个分布 $\mathbb{P}$ 的总概率质量的 $1-\varepsilon$ 的一部分，那么最坏情况损失 $\sup _{z \in \mathcal{Z}} \ell(x, z)$ 保证超过在 $\mathbb{P}$ 下 $\ell(x, Z)$ 的 $(1-\varepsilon)$-分位数。这可以通过利用 $\mathbb{P}$ 的先验结构信息或统计数据来实现。例如，如果 $\mathbb{P}(Z \in \mathcal{Z}) \geq 1-\varepsilon$（确定成立），则可以通过构造一个合适大小的半空间和椭球体的交集来满足这一条件，且如果 $Z$ 在 $\mathbb{P}$ 下具有独立的、对称的、单峰的和/或次高斯分量（Bertsimas and Sim 2004, Janak, Lin and Floudas 2007, BenTal et al. 2009, Li, Ding and Floudas 2011, Bertsimas, den Hertog and Pauphilet 2021）。或者，如果 $\mathcal{Z}$ 是通过使用统计假设检验（Postek, den Hertog and Melenberg 2016, Bertsimas, Gupta and Kallus 2018b,a）、分位数估计（Hong, Huang and Lam 2021）或基于学习的方法（Han, Shang and Huang 2021, Goerigk and Kurtz 2023, Wang, Becker, Van Parys and Stellato 2023）从 $\mathbb{P}$ 的独立样本中构建的，则可能高置信度成立。

### 2.1.2. Markov 模糊集合

Markov 不等式为一个非负单变量随机变量 $Z$ 提供了一个上界，该随机变量的均值为 $\mu \geq 0$，且其超过一个正阈值 $\tau>0$ 的概率。正式地，它表明对于模糊集合 $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}\left(\mathbb{R}_{+}\right): \mathbb{E}_{\mathbb{P}}[Z]=\mu\right\}$ 中的每个可能的 $Z$ 的概率分布，$\mathbb{P}(Z \geq \tau) \leq \mu / \tau$。如果 $\mu \leq \tau$，那么 Markov 不等式是紧的，即存在一个概率分布 $\mathbb{P}^{\star} \in \mathcal{P}$，使得该不等式成立为等式。实际上，分布 $\mathbb{P}^{\star}=(1-\mu / \tau) \delta_0 + \mu / \tau \delta_\tau$，其中 $\delta_z$ 是 Dirac 分布，它将点质量放置在 $z \in \mathbb{R}$ 上，属于 $\mathcal{P}$ 并满足 $\mathbb{P}(Z \geq \tau) = \mu / \tau$。这些洞察表明 $\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{P}(Z \geq \tau) = \mu / \tau$，并且当 $\mu \leq \tau$ 时，最大值由 $\mathbb{P}^{\star}$ 达到。因此，Markov 上界可以被解释为一个 DRO 问题的最优值。因此，通常将 $\mathcal{P}$ 称为 Markov 模糊集合。更一般地，我们将与闭合支持集 $\mathcal{Z} \subseteq \mathbb{R}^d$ 和均值向量 $\mu \in \mathbb{R}^d$ 对应的 Markov 模糊集合定义为形式为

$$
\mathcal{P}=\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathrm{P}}[Z]=\mu\right\} .
$$

因此，Markov 模糊集合（2.2）包含所有支持在 $\mathcal{Z}$ 上的分布，这些分布具有相同的均值向量 $\mu$。然而，这些分布可能具有截然不同的形状和高阶矩。Markov 模糊集合上的最坏情况期望有时被用作在随机规划中对期望成本函数的有效计算上界。如果成本函数在不确定问题参数中是凹的，则这些最坏情况期望与 Jensen 不等式紧密相关（Jensen 1906）；详见第 6.1 节。另一方面，如果成本函数是凸的，并且 $\mathcal{Z}$ 是一个多面体，则这些最坏情况期望与 Edmundson-Madansky 不等式相关（Edmundson 1956，Madansky 1959）；详见第 6.2 节。



### 2.1.3. Chebyshev 模糊集合

Chebyshev 不等式提供了一个上界，说明一个具有有限均值 $\mu \in \mathbb{R}$ 和方差 $\sigma^2>0$ 的单变量随机变量 $Z$ 偏离其均值超过 $k>0$ 个标准差的概率。正式地，它表明对于模糊集合 $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathbb{R}): \mathbb{E}_{\mathbb{P}}[Z]=\mu, \mathbb{E}_{\mathbb{P}}\left[Z^2\right]=\sigma^2+\mu^2\right\}$ 中的每个可能的 $Z$ 的概率分布，$\mathbb{P}(|Z-\mu| \geq k \sigma) \leq 1 / k^2$。如果 $k \geq 1$，则 Chebyshev 不等式是紧的。实际上，可以很容易验证分布

$$
\mathbb{P}^{\star}=\frac{1}{2 k^2} \delta_{\mu-k \sigma}+\left(1-\frac{1}{k^2}\right) \delta_\mu+\frac{1}{2 k^2} \delta_{\mu+k \sigma}
$$

是 $\mathcal{P}$ 的一个元素，并且满足 $\mathbb{P}(|Z-\mu| \geq k \sigma)=1 / k^2$。这些洞察表明，Chebyshev 上界可以被解释为一个 DRO 问题的最优值。因此，通常将 $\mathcal{P}$ 称为 Chebyshev 模糊集合。更一般地，我们将与闭合支持集 $\mathcal{Z} \subseteq \mathbb{R}^d$、均值向量 $\mu \in \mathbb{R}^d$ 和二阶矩阵 $M \in \mathbb{S}_{+}^d, M \geq \mu \mu^{\top}$ 对应的 Chebyshev 模糊集合定义为

$$
\mathcal{P}=\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{E}_{\mathrm{P}}[Z]=\mu, \mathbb{E}_{\mathrm{P}}\left[Z Z^{\top}\right]=M\right\}
$$

因此，Chebyshev 模糊集合（2.3）包含所有支持在 $\mathcal{Z}$ 上的分布，这些分布具有相同的均值向量 $\mu$ 和二阶矩阵 $M$（因此也具有相同的协方差矩阵 $\Sigma=M-\mu \mu^{\top} \in \mathbb{S}_{+}^d$）。然而，这些分布可能具有截然不同的形状和高阶矩。

Chebyshev 模糊集合（2.3）捕捉了与多变量 Chebyshev 不等式相关的分布信息（Lal 1955，Marshall 和 Olkin 1960，Tong 1980，Rujeerapaiboon, Kuhn 和 Wiesemann 2018）。在运筹学中，Chebyshev 模糊集合自 Scarf（1958）关于分布鲁棒新闻供应商的开创性工作以来被广泛使用，该工作通常被认为是 DRO 的第一篇论文。从那时起，许多与 Chebyshev 模糊集合相关的 DRO 模型在新闻供应商和投资组合选择问题的背景下涌现。这些模型涉及各种不同的决策标准，如期望值（Gallego 和 Moon 1993，Natarajan 和 Linyi 2007，Popescu 2007），风险价值（El Ghaoui 等 2003，Xu, Caramanis 和 Mannor 2012b，Zymler, Kuhn 和 Rustem 2013a,b，Rujeerapaiboon, Kuhn 和 Wiesemann 2016，Yang 和 Xu 2016，Zhang, Jiang 和 Shen 2018），条件风险价值（Natarajan, Sim 和 Uichanco 2010，Chen, He 和 Zhang 2011，Zymler 等 2013b，Hanasusanto, Kuhn, Wallace 和 Zymler 2015a），谱风险度量（Li 2018）和失真风险度量（Cai, Li 和 Mao 2023，Pesenti, Wang 和 Wang 2024），以及最大最小遗憾标准（Yue, Chen 和 Wang 2006，Perakis 和 Roels 2008）。2008)。除此之外，切比雪夫模糊集在期权和股票定价（Bertsimas 和 Popescu 2002）、统计学和机器学习（Lanckriet, El Ghaoui, Bhattacharyya 和 Jordan 2001, 2002, Strohmann 和 Grudic 2002, Huang, Yang, King, Lyu 和 Chan 2004, Bhattacharyya 2004, Farnia 和 Tse 2016, Nguyen, Shafieezadeh-Abadeh, Yue, Kuhn 和 Wiesemann 2019, Rontsis, Osborne 和 Goulart 2020）、随机规划（Birge 和 Wets 1986, Dulá 和 Murthy 1992, Dokov 和 Morton 2005, Bertsimas, Doan, Natarajan 和 Teo 2010, Natarajan, Teo 和 Zheng 2011）、控制（Van Parys, Kuhn, Goulart 和 Morari 2015, Yang 2018, Xin 和 Goldberg 2021, 2022）、电力系统运行（Xie 和 Ahmed 2017, Zhao 和 Jiang 2017）、复杂网络分析（Van Leeuwaarden 和 Stegehuis 2021, Brugman, Van Leeuwaarden 和 Stegehuis 2022）、排队系统（van Eekelen, Hanasusanto, Hasenbein 和 van Leeuwaarden 2023）、医疗保健（Mak, Rong 和 Zhang 2015, Shehadeh, Cohn 和 Jiang 2020）、极端事件分析（Lam 和 Mottet 2017）等方面都得到了广泛应用。


2.1.4. 切比雪夫模糊集与不确定的矩

在已知 $\mathbb{P}$ 的一阶和二阶矩时，使用切比雪夫模糊集是合适的，但所有更高阶的矩是未知的。然而，在实际中，即使是已知的一阶和二阶矩也从来不能绝对确定。相反，它们必须从统计数据中估计，因此会受到估计误差的影响。这促使 El Ghaoui 等人（2003）引入了具有不确定矩的切比雪夫模糊集，可以表示为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}):\left(\mathrm{E}_{\mathrm{P}}[Z], \mathrm{B}_{\mathrm{P}}\left[Z Z^{\top}\right]\right) \in \mathcal{F}\right\} .
$$

这里，$\mathcal{F} \subseteq \mathbb{R}^d \times \mathbb{S}_{+}^d$ 是一个凸集，用于捕捉矩的不确定性。显然，$\mathcal{P}$ 可以表示为一系列切比雪夫模糊集的并集，即：

$$
\mathcal{P}=\bigcup_{(\mu, M) \in \mathcal{F}}\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{E}_{\mathrm{P}}[Z]=\mu, \mathbb{E}_{\mathrm{P}}\left[Z Z^{\top}\right]=M\right\} .
$$

注意，具有不确定矩的切比雪夫模糊集涵盖了支持集模糊集、马尔可夫模糊集和切比雪夫模糊集作为特例。它们通过设置 $\mathcal{F}=\mathbb{R}^d \times \mathbb{S}_{+}^d, \mathcal{F}=\{\mu\} \times \mathbb{S}_{+}^d$ 和 $\mathcal{F}=\{\mu\} \times\{M\}$ 分别恢复。

El Ghaoui 等人（2003）通过以下盒子捕捉矩的不确定性：

$$
\mathcal{F}=\left\{(\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: \underline{\mu} \leq \mu \leq \bar{\mu}, \underline{M} \leq M \leq \bar{M}\right\}
$$

该框由矩的界限参数化，其中 $\underline{\mu}, \bar{\mu} \in \mathbb{R}^d$ 和 $\underline{M}, \bar{M} \in \mathbb{S}_{+}^d$。

给定对未知均值向量和协方差矩阵的噪声估计 $\hat{\mu}$ 和 $\hat{\Sigma}$，Delage 和 Ye（2010）提出了以下模糊集：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \begin{array}{l}
\left(\mathbb{E}_{\mathrm{P}}[Z]-\hat{\mu}\right)^{\top} \hat{\Sigma}^{-1}\left(\mathbb{E}_{\mathrm{P}}[Z]-\hat{\mu}\right) \leq \gamma_1 \\
\mathbb{E}_{\mathrm{P}}\left[(Z-\hat{\mu})(Z-\hat{\mu})^{\top}\right] \leq \gamma_2 \hat{\Sigma}
\end{array}\right\} .
$$

通过构造，$\mathcal{P}$ 包含所有分布在 $\mathcal{Z}$ 上，其一阶矩位于以 $\hat{\mu}$ 为中心的椭球内，二阶矩（相对于 $\hat{\mu}$）位于以 $\gamma_2 \hat{\Sigma}$ 为顶点的半正定锥内。一个基本的计算揭示了：

$$
\mathbb{E}_{\mathrm{P}}\left[(Z-\hat{\mu})(Z-\hat{\mu})^{\top}\right]=\mathrm{E}_{\mathrm{P}}\left[Z Z^{\top}\right]-\mathbb{B}_{\mathrm{P}}[Z] \hat{\mu}^{\top}-\hat{\mu} \mathrm{E}_{\mathrm{P}}[Z]^{\top}+\hat{\mu} \hat{\mu}^{\top}
$$
因此，$\mathbb{P}$ 可以视为一个具有不确定矩的切比雪夫模糊集合。实际上，如果我们将矩不确定集定义为

$$\mathcal{F}=\left\{(\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: \begin{array}{l}(\mu-\hat{\mu})^{\top} \hat{\Sigma}(\mu-\hat{\mu}) \leq \gamma_1 \\ M-\mu \hat{\mu}^{\top}-\hat{\mu} \mu^{\top}+\hat{\mu} \hat{\mu}^{\top} \leq \gamma_2 \hat{\Sigma}\end{array}\right\}.
$$

Delage 和 Ye（2010）表明，如果 $\rho$ 和 $\sum$ 被设置为样本均值和从有限数量的独立样本中提取的协方差矩阵，那么可以调整集参数 $\gamma_1 \geq 0$ 和 $\gamma_2 \geq 1$，以确保 $\mathbb{P}$ 属于 $\mathcal{P}$，并具有任何期望的置信度。

具有不确定矩的切比雪夫和马尔可夫模糊集合在多个领域找到了应用，涉及的领域包括控制（Naka, Jiang 和 Shen 2021）、积分随机规划（Bertamas, Natarajan 和 Teo 2004）、事件分析（Bai, Lam 和 Doung 2023）、机制设计和定价（Bertsimas 2024）等众多领域。

不确定集$\mathcal{F}$通常对应于某个名义均值-协方差对$(\hat{\mu}, \hat{\Sigma})$的邻域，依据某种偏差度量。例如，矩阵范数，如Frobenius范数、谱范数或核范数（Bernstein 2009, § 9），提供了自然的度量方法来量化协方差矩阵的相似性。两个均值-协方差对$(\mu, \Sigma)$和$(\hat{\mu}, \hat{\Sigma})$之间的偏差也可以定义为正态分布$\mathcal{N}(\mu, \Sigma)$和$\mathcal{N}(\hat{\mu}, \hat{\Sigma})$之间的偏差，依据某种概率度量或信息论上的散度，如Kullback-Leibler散度（Kullback 1959）、Fisher-Rao距离（Atkinson 和 Mitchell 1981）或其他谱散度（Zorzi 2014）。

正如我们将在第2.3节中更详细地讨论的那样，两个正态分布$\mathcal{N}(\mu, \Sigma)$和$\mathcal{N}(\hat{\mu}, \hat{\Sigma})$之间的2-Wasserstein距离与其底层均值-协方差对$(\mu, \Sigma)$和$(\hat{\mu}, \hat{\Sigma})$之间的Gelbrich距离一致。接下来，我们首先给出Gelbrich距离的正式定义，然后举例说明如何用它来定义一个矩 moment 不确定集$\mathcal{F}$。

定义 2.1 (Gelbrich 距离)。在 $\mathbb{R}^d \times \mathbb{S}_{+}^d$ 中，两个均值协方差对 $(\mu, \Sigma)$ 和 $(\hat{\mu}, \hat{\Sigma})$ 之间的 Gelbrich 距离定义为

$$
\mathrm{G}((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma}))=\sqrt{\|\mu-\hat{\mu}\|_2^2+\operatorname{Tr}\left(\Sigma+\hat{\Sigma}-2\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}\right)} .
$$

Gelbrich 距离是非负的、对称的和次可加的，且当且仅当 $(\mu, \Sigma) = (\hat{\mu}, \hat{\Sigma})$ 时它为零。因此，它表示了 $\mathbb{R}^d \times \mathbb{S}_{+}^d$ 上的一个度量（Givens 和 Short 1984, 第239页）。当 $\mu = \hat{\mu}$ 时，Gelbrich 距离退化为 $\Sigma$ 和 $\hat{\Sigma}$ 之间的 Bures 距离，后者作为量子信息理论中密度矩阵之间相异性的度量。Bures 距离已知能在正半定矩阵空间上诱导一个黎曼度量（Bhatia, Jain 和 Lim 2018, 2019）。当 $\Sigma$ 和 $\hat{\Sigma}$ 可以同时对角化时，它们的 Bures 距离与它们谱之间的 Hellinger 距离一致。Hellinger 距离与信息理论中广泛使用的 Fisher-Rao 度量密切相关（Liese 和 Vajda 1987）。尽管 Gelbrich 距离是非凸的，但平方的 Gelbrich 距离在其两个参数中是联合凸的。这是以下命题的直接结果，该命题可以在（Olkin 和 Pukelsheim 1982, Dowson 和 Landau 1982, Givens 和 Shortt 1984, Panaretos 和 Zemel 2020）中找到。

命题 2.2 (Gelbrich 距离的 SDP 表示)。对于任意均值协方差对 $(\mu, \Sigma)$ 和 $(\hat{\mu}, \hat{\Sigma})$ 在 $\mathbb{R}^d \times \mathbb{S}_{+}^d$ 中，我们有

$$
\mathrm{G}^2((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma}))=\left\{\begin{array}{cl}
\min _{C \in \mathbb{R}^{d \times d}} & \|\mu-\hat{\mu}\|_2^2+\operatorname{Tr}(\Sigma+\hat{\Sigma}-2 C) \\
\text { s.t. } & {\left[\begin{array}{cc}
\Sigma & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0 .}
\end{array}\right.
$$

证明：在证明过程中，我们保持 $\mu, \hat{\mu}$ 和 $\Sigma$ 不变，并将 $\hat{\Sigma}$ 视为参数。我们还使用 $f(\hat{\Sigma})$ 作为 (2.5) 左侧的简写，$g(\hat{\Sigma})$ 作为 (2.5) 右侧的简写。基础操作显示

$$
g(\hat{\Sigma})=\|\mu-\hat{\mu}\|_2^2+\operatorname{Tr}(\Sigma+\hat{\Sigma})-\left\{\begin{array}{cl}
\max _{C \in \mathbb{R}^{d \times d}} & \operatorname{Tr}(2 C) \\
\text { s.t. } & {\left[\begin{array}{cc}
\Sigma & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0 .}
\end{array}\right.
$$

(2.6) 中的最大化问题是以下最小化问题的对偶问题。

$$
\begin{array}{cl}
\inf _{A_{11}, A_{22} \in \mathbb{S}^d} & \operatorname{Tr}\left(A_{11} \Sigma\right)+\operatorname{Tr}\left(A_{22} \hat{\Sigma}\right) \\
\text { s.t. } & {\left[\begin{array}{cc}
A_{11} & I_d \\
I_d & A_{22}
\end{array}\right] \geq 0}
\end{array}
$$

由于 $A_{11}=A_{22}=2 I_d$ 构成了对偶问题的 Slater 点（Ben-Tal 和 Nemirovski 2001, Theorem 2.4.1），因此强对偶性成立。Slater 点的存在进一步意味着 (2.6) 中的原始最大化问题以及 (2.5) 中的最小化问题是可解的。根据 (Bernstein 2009, Corollary 8.2.2)，为了保证对偶可行性，$A_{11}$ 和 $A_{22}$ 必须是正定的。因此，它们是可逆的。我们可以使用 Schur 互补论证（Ben-Tal 和 Nemirovski 2001, Lemma 4.2.1）将对偶问题简化为

$$
\inf _{A_{11} \geq A_{22}^{-1}>0} \operatorname{Tr}\left(A_{11} \Sigma\right)+\operatorname{Tr}\left(A_{22} \hat{\Sigma}\right)=\inf _{A_{22}>0} \operatorname{Tr}\left(A_{22}^{-1} \Sigma\right)+\operatorname{Tr}\left(A_{22} \hat{\Sigma}\right),
$$
其中等式成立是因为 $\Sigma \geq 0$。得到的最小化问题的最优值在 $\hat{\Sigma}$ 中是凹的，并且是上半连续的，因为它构成了 $\hat{\Sigma}$ 的仿射函数的逐点下确界。因此，$g(\hat{\Sigma})$ 是凸的并且下半连续的。现在我们展示，如果 $\hat{\Sigma} > 0$，那么 (2.7) 中对 $A_{22}$ 的凸最小化问题可以解析求解。为此，我们构造一个满足问题一阶最优性条件的正定矩阵 $A_{22}^{\star}$：

$$
\hat{\Sigma}-A_{22}^{-1} \Sigma A_{22}^{-1}=0 \quad \Longleftrightarrow \quad A_{22} \hat{\Sigma} A_{22}-\Sigma=0 .
$$

事实上，右侧的二次方程两边同时与 $\hat{\Sigma}^{\frac{1}{2}}$ 相乘，得到等价方程 $\left(\hat{\Sigma}^{\frac{1}{2}} A_{22} \hat{\Sigma}^{\frac{1}{2}}\right)^2=\hat{\Sigma}^{\frac{1}{2}} \Sigma^{\frac{1}{2}}$。由于 $\hat{\Sigma} > 0$，该方程唯一解为 $A_{22}^{\star}=\hat{\Sigma}^{-\frac{1}{2}}\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}} \hat{\Sigma}^{-\frac{1}{2}}$。将 $A_{22}^{\star}$ 代入 (2.7) 中，结果显示对偶最小化问题的最优值为 $2 \operatorname{Tr}\left(\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}\right)$。然后将该值代入 (2.6) 中，显示当 $\hat{\Sigma} > 0$ 时，$g(\hat{\Sigma})=f(\hat{\Sigma})$。

接下来我们展示，如果 $\hat{\Sigma}$ 是奇异的，那么 $g(\hat{\Sigma}) = f(\hat{\Sigma})$。为此，我们回顾 (Nguyen, Shafieezadeh-Abadeh, Kuhn 和 Mohajerin Esfahani 2023, Lemma A.2) 中的结论，即矩阵平方根在 $\mathbf{S}_{+}^d$ 上是连续的，这意味着 $f(\hat{\Sigma})$ 在 $\mathbb{S}_{+}^d$ 上是连续的。对于任何奇异的 $\hat{\Sigma} \geq 0$，我们有

$$
f(\hat{\mathbf{\Sigma}})=\liminf _{\hat{\mathbf{\Sigma}}^{\prime} \rightarrow \hat{\mathbf{\Sigma}}, \hat{\mathbf{\Sigma}}^{\prime}>0} f\left(\hat{\mathbf{\Sigma}}^{\prime}\right)=\liminf _{\hat{\mathbf{\Sigma}}^{\prime} \rightarrow \hat{\mathbf{\Sigma}}, \hat{\Sigma}^{\prime}>0} g\left(\hat{\Sigma}^{\prime}\right)=g(\hat{\mathbf{\Sigma}}) .
$$

这里，第一个等式利用了 $f$ 的连续性，第二个等式成立是因为对于每个 $\hat{\Sigma}^{\prime} > 0$，$f\left(\hat{\Sigma}^{\prime}\right)=g\left(\hat{\Sigma}^{\prime}\right)$。第三个等式来自于 $g$ 的凸性和下半连续性，这意味着极限下确界不能小于或大于 $g(\hat{\Sigma})$，分别地。由此，证明完毕。

命题 2.2 显示，平方的 Gelbrich 距离与一个可解的半正定程序的最优值相同。这使得 Gelbrich 距离在计算中具有吸引力。作为副产品，命题 2.2 的证明揭示了平方 Gelbrich 距离在其定义域上是凸的并且连续的。

根据 Nguyen、Shafiee、Filipovic 和 Kuhn (2021) 的方法，我们现在可以将 Gelbrich 模糊集合作为不确定矩的 Chebyshev 模糊集合 (2.4) 的一个实例。相应的矩不确定集合如下所示：

$$
\mathcal{F}=\left\{(\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: \begin{array}{l}
\exists \Sigma \in \mathbb{S}_{+}^d \text { with } M=\Sigma+\mu \mu^{\top}, \\
\mathrm{G}((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma})) \leq r
\end{array}\right\},
$$

其中 $(\hat{\mu}, \hat{\Sigma})$ 是一个名义均值-协方差对，半径 $r \geq 0$ 是一个可调节的大小参数。下面我们将 $\mathcal{F}$ 称为 Gelbrich 不确定集合。下一个命题建立了 $\mathcal{F}$ 的基本拓扑和计算性质。

命题 2.3 (Gelbrich 不确定集合)。不确定集合 $\mathcal{F}$ 如 (2.8) 所定义，是凸的并且是紧的。此外，它具有以下半正定表示：

$$
\mathcal{F}=\left\{\begin{array}{ll}
(\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: & \exists C \in \mathbb{R}^{d \times d}, U \in \mathbb{S}_{+}^d \text { with } \\
\|\hat{\mu}\|_2^2-2 \mu^{\top} \hat{\mu}+\operatorname{Tr}(M+\hat{\Sigma}-2 C) \leq r^2, \\
{\left[\begin{array}{cc}
M-U & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0,\left[\begin{array}{cc}
U & \mu \\
\mu^{\top} & 1
\end{array}\right] \geq 0}
\end{array}\right\} .
$$

证明。该证明利用了命题 2.2 中建立的平方 Gelbrich 距离的半正定表示。首先注意到，如果 $M = \Sigma + \mu \mu^{\top}$，那么

$$
\|\mu - \hat{\mu}\|_2^2 + \operatorname{Tr}(\Sigma + \hat{\Sigma} - 2 C) = \|\hat{\mu}\|_2^2 - 2 \mu^{\top} \hat{\mu} + \operatorname{Tr}(M + \hat{\Sigma} - 2 C) .
$$

根据命题 2.2，Gelbrich 不确定集合 $\mathcal{F}$ 可以表示为

$$
\mathcal{F} = \left\{ \begin{array}{ll}
& \exists C \in \mathbb{R}^{d \times d} \text{ with } \\
& \|\hat{\mu}\|_2^2 - 2 \mu^{\top} \hat{\mu} + \operatorname{Tr}(M + \hat{\Sigma} - 2 C) \leq r^2, \\
{\left[\begin{array}{cc}
M - \mu \mu^{\top} & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0}
\end{array} \right\} .
$$

一个标准的 Schur 余补推论进一步揭示

$$
\left[\begin{array}{cc}
M - \mu \mu^{\top} & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0 \Longleftrightarrow \exists U \in \mathbb{S}_{+}^d \text{ with } \left[\begin{array}{cc}
M - U & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0, \left[\begin{array}{cc}
U & \mu \\
\mu^{\top} & 1
\end{array}\right] \geq 0 .
$$

因此，Gelbrich 不确定集合 $\mathcal{F}$ 具有命题中给出的半正定表示。由于该表示通过锥不等式定义的集合的投影表达了 $\mathcal{F}$，因此其凸性显而易见。

接下来需要证明的是，$\mathcal{F}$ 是紧的。为此，我们定义

$$
\mathcal{V} = \left\{ (\mu, \Sigma) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: \mathrm{G}((\mu, \Sigma), (\hat{\mu}, \hat{\Sigma})) \leq r \right\}
$$

作为关于 $(\hat{\mu}, \hat{\Sigma})$ 的半径为 $r$ 的球，使用 Gelbrich 距离来度量。注意到 $\mathcal{F} = f(\mathcal{V})$，其中变换 $f: \mathbb{R}^d \times \mathbb{S}_{+}^d \rightarrow \mathbb{R}^d \times \mathbb{S}_{+}^d$ 定义为 $f(\mu, \Sigma) = \left(\mu, \Sigma + \mu \mu^{\top}\right)$。现在我们将证明 $\mathcal{V}$ 是紧的。由于 $f$ 是连续的，并且紧性在连续变换下保持，因此这将直接推导出 $\mathcal{F}$ 是紧的。显然，$\mathcal{V}$ 是闭的，因为 Gelbrich 距离是连续的。为了证明 $\mathcal{V}$ 也是有界的，固定任意 $(\mu, \Sigma) \in \mathcal{V}$。根据 Gelbrich 距离的定义，我们有 $\|\mu - \hat{\mu}\| \leq r^2$。此外，我们可以得到

$$
\begin{aligned}
\operatorname{Tr}\left(\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}\right) &= \max_{C \in \mathbb{R}^{d \times d}} \left\{ \operatorname{Tr}(C): \left[\begin{array}{cc}
\Sigma & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0 \right\} \\
&\leq \max_{C \in \mathbb{R}^{d \times d}} \left\{ \operatorname{Tr}(C): C_{ij}^2 \leq \Sigma_{ii} \hat{\Sigma}_{jj} \ \forall i,j \in[d] \right\} \leq \sqrt{\operatorname{Tr}(\Sigma) \operatorname{Tr}(\hat{\Sigma})},
\end{aligned}
$$

其中，等式已在命题 2.2 的证明中建立。两个不等式来自于线性矩阵不等式的放松，它利用了一个观察结果，即一个正半定矩阵的所有二阶主子式是非负的，并且使用了 Cauchy-Schwarz 不等式。因此，$\Sigma$ 满足

$$
\left.\left(\operatorname{Tr}(\Sigma)^{\frac{1}{2}} - \operatorname{Tr}(\hat{\Sigma})^{\frac{1}{2}}\right)^2 \leq \operatorname{Tr}\left(\Sigma + \hat{\Sigma} - 2\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}\right)\right) \leq r^2,
$$

其中第二个不等式成立是因为 $(\mu, \Sigma) \in \mathcal{V}$。因此，我们可以得出结论：$\operatorname{Tr}(\Sigma) \leq \left(r + \operatorname{Tr}(\hat{\Sigma})^{\frac{1}{2}}\right)^2$，进而得到 $0 \leq \Sigma \leq \left(r + \left(\operatorname{Tr}(\hat{\Sigma})\right)^{\frac{1}{2}}\right)^2 I_d$。总之，我们已经证明了 $\mu$ 和 $\Sigma$ 都属于有界集合。由于选择的 $(\mu, \Sigma) \in \mathcal{V}$ 是任意的，这证明了 $\mathcal{V}$ 确实是有界的，因此是紧的。得证。

命题 2.2 显示了不确定性集合 $\mathcal{F}$ 是凸的。这一点令人惊讶，因为 $\mathcal{F} = f(\mathcal{V})$，其中 Gelbrich 球 $\mathcal{V}$ 在均值-协方差对的空间中是凸的，得益于命题 2.2，而 $f$ 是一个二次双射。事实上，通常只有在仿射变换下，凸性才会被保留。

Gelbrich 不确定性集是由 Nguyen 等人（2021）在稳健投资组合优化的背景下引入的。它们还在机器学习（Bui, Nguyen 和 Nguyen 2022, Vu, Tran, Yue 和 Nguyen 2021, Nguyen, Bui 和 Nguyen 2022a）、估计（Nguyen 等人 2023）、滤波（Shafieezadeh-Abadeh, Nguyen, Kuhn 和 Mohajerin Esfahani 2018, Kargin, Hajar, Malik 和 Hassibi 2024b）以及控制（McAllister 和 Mohajerin Esfahani 2023, Al Taha, Yan 和 Bitar 2023, Hajar, Kargin 和 Hassibi 2023, Hakobyan 和 Yang 2024, Taşkesen, Iancu, Koçyiğit 和 Kuhn 2024, Kargin, Hajar, Malik 和 Hassibi 2024a,c,d）中找到了应用。



### 2.1.5. 均值-离散不确定性集

如果 $\mathcal{K} \subseteq \mathbb{R}^k$ 是一个适当的凸锥，且 $v_1, v_2 \in \mathbb{R}^k$，则不等式 $v_1 \leq \mathcal{K} v_2$ 表示 $v_2 - v_1 \in \mathcal{K}$。此外，函数 $G: \mathbb{R}^m \rightarrow \mathbb{R}^k$ 被称为 $\mathcal{K}$-凸的，如果满足

$$
G\left(\theta v_1 + (1 - \theta) v_2\right) \leq \mathcal{K} \theta G\left(v_1\right) + (1 - \theta) G\left(v_2\right) \quad \forall v_1, v_2 \in \mathbb{R}^m, \forall \theta \in [0,1] .
$$

对应于一个凸闭支持集 $\mathcal{Z} \subseteq \mathbb{R}^d$，均值向量 $\mu \in \mathbb{R}^d$，一个 $\mathcal{K}$-凸的离散函数 $G: \mathbb{R}^m \rightarrow \mathbb{R}^k$ 和一个离散界限 $g \in \mathbb{R}^k$，均值-离散不确定性集定义为

$$
\mathcal{P} = \left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{B}_{\mathrm{P}}[Z] = \mu, \mathbb{E}_{\mathrm{P}}[G(Z)] \leq \mathcal{K} g\right\}
$$

均值-离散不确定性集具有很强的表达能力，即它能够建模未知概率分布的各种典型特征。例如，如果 $\|\cdot\|$ 是 $\mathbb{R}^d$ 上的一个范数，$G(z) = \|z - \mu\|$ 是通常意义上的凸函数，且 $g = \sigma \in \mathbb{R}_{+}$，则所有的分布 $\mathbb{P} \in \mathcal{P}$ 的均值绝对偏差都被 $\sigma$ 有界。或者，如果 $G(z) = (z - \mu)(z - \mu)^{\top}$ 是 $\mathbb{S}_{+}^d$-凸的，且 $g = \Sigma \in \mathbb{S}_{+}^d$，则 $\mathcal{P}$ 退化为具有时刻不确定性的 Chebyshev 不确定性集。具体来说，任何 $\mathbb{P} \in \mathcal{P}$ 的协方差矩阵都被 $\Sigma$ 限制在 Loewner 顺序中。Wiesemann, Kuhn 和 Sim (2014) 表明，包含 $d$ 维随机向量 $Z$ 分布的不确定性集 $\mathcal{P}$ 与提升的不确定性集紧密相关：

$$
\mathcal{Q} = \left\{\mathbb{Q} \in \mathcal{P}(\mathcal{C}): \mathbb{B}_{\mathrm{Q}}[Z] = \mu, \mathbb{E}_{\mathrm{Q}}[U] = g\right\}
$$

其中支持集 $\mathcal{C} = \left\{(z, u) \in \mathcal{Z} \times \mathbb{R}^k: G(z) \leq_{\mathcal{K}} u\right\}$，包含 $Z$ 和一个辅助的 $m$ 维随机向量 $U$ 的联合分布。实际上，可以证明 $\mathcal{P} = \left\{Q_z: Q \in \mathcal{Q}\right\}$，其中 $Q_z$ 表示 $Q$ 下的 $Z$ 的边际分布。由于损失函数仅依赖于 $Z$ 而不依赖于 $U$，这一推理意味着（1.2）中的内层最坏情况期望问题满足

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{B}_{\mathrm{P}}[\ell(x, Z)] = \sup _{\mathrm{Q} \in \mathcal{Q}} \mathbb{E}_{\mathrm{Q}}[\ell(x, Z)] .
$$

因此，可以用提升的不确定性集 $\mathcal{Q}$ 替代原始的不确定性集 $\mathcal{P}$。这是有用的，因为 $\mathcal{Q}$ 是一个简单的 Markov 不确定性集，只指定联合随机向量 $(Z, U)$ 的支持集 $\mathcal{C}$ 和均值 $(\mu, g)$。此外，可以证明 $\mathcal{Z}$ 是凸的，因为 $\mathcal{Z}$ 本身是凸的，且 $G$ 是 $\mathcal{K}$-凸的。总之，具有均值-离散不确定性集（形式为（2.9））的 DRO 问题可以系统地转化为具有 Markov 不确定性集的 DRO 问题。

一种更一般的均值-离散不确定性集可以用来塑造 $Z$ 在 $\mathbb{P}$ 下的矩生成函数。具体来说，Chen、Sim 和 Xu（2019）引入了熵支配不确定性集：

$$
\mathcal{P} = \left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}[Z] = \mu, \log \left(\mathbb{E}_{\mathbb{P}}\left[\exp \left(\theta^{\top}(Z-\mu)\right)\right]\right) \leq g(\theta) \forall \theta \in \mathbb{R}^d \right\},
$$

其中 $g: \mathbb{R}^d \rightarrow \mathbb{R}$ 是一个凸且连续二次可微的函数，满足 $g(0) = 0$ 且 $\nabla g(0) = 0$。由 $\theta$ 参数化的约束对聚合生成函数（即对中心化随机变量 $Z-\mu$ 的对数矩生成函数）施加了连续的上界。$g$ 的选择决定了不确定性集中包含的具体分布类别。例如，如果 $g(\theta) = \sigma^2 \theta^{\top} \theta / 2$（其中 $\sigma > 0$），则该不确定性集只包含方差代理为 $\sigma^2$ 的亚高斯分布。亚高斯分布是尾部受高斯分布尾部限制的概率分布。它们在概率论和统计学中扮演着重要角色，特别是在浓度不等式和高维现象的研究中（Vershynin 2018，Wainwright 2019）。

熵支配不确定性集对 $\mathbb{P}$ 强加了无限多的约束。Chen 等人（2019）表明，在该不确定性集上的最坏情况期望问题可以重新表述为半无限锥程序。他们提出了一种切平面算法，以高效地求解这些锥程序。熵支配不确定性集还在非线性和 PDE 约束的 DRO 问题研究中得到了应用（Milz 和 Ulbrich 2020，2022）。Chen、Fu、Si、Sim 和 Xiong（2023）考虑了广义的熵支配不确定性集。


### 2.1.6. Higher-order Moment Ambiguity Sets

Markov 和 Chebyshev 不确定性集仅对 $\mathbb{P}$ 的一阶和/或二阶矩施加条件。具有这种不确定性集的 DRO 问题通常是可处理的。在本节中，我们简要讨论对 $\mathbb{P}$ 的高阶（多项式）矩施加条件的矩不确定性集，这通常会导致 NP-hard 的 DRO 问题（Popescu 2005，第 4.5 和 4.6 命题）。

现在假设 $\mathcal{Z}$ 是一个闭半代数集，可以表示为有限多个多项式不等式的可行集。此外，定义在 $z \in \mathbb{R}^d$ 中的阶数为 $\alpha \in \mathbb{Z}_{+}^d$ 的单项式为函数 $\prod_{i=1}^d z_i^{\alpha_i}$，我们简洁地表示为 $z^\alpha$。由有限索引集 $\mathcal{A} \subseteq \mathbb{Z}_{+}^d$ 和矩界限 $m_\alpha \in \mathbb{R}, \alpha \in \mathcal{A}$ 诱导的高阶矩不确定性集为

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}\left[Z^\alpha\right] \leq m_\alpha \forall \alpha \in \mathcal{A}\right\}
$$

因此，评估多项式函数（或半代数集的特征函数）在所有分布 $\mathcal{P}$ 上的最坏情况期望就等同于求解一个广义的矩问题。这个矩问题及其对偶问题构成了半无限线性规划，可以被重新构造为某些矩锥和相应的非负多项式对偶锥上的有限维锥优化问题（Karlin 和 Studden 1966，Zuluaga 和 Pena 2005）。尽管一般情况下是 NP-hard，这些锥优化问题可以通过多项式优化工具（Parrilo 2000, 2003，Lasserre 2001, 2009）逼近为越来越紧的可处理半定规划序列。这种通用技术为不确定性集 $\mathcal{P}$ 提供了最坏情况期望界限和广义 Chebyshev 不等式（Bertsimas 和 Sethuraman 2000，Lasserre 2002，Popescu 2005，Lasserre 2008）。此外，它还为最坏情况风险度量提供了紧的界限（Natarajan，Pachamanova 和 Sim 2009a）。


## 2.2. $\phi$-Divergence Ambiguity Sets

概率分布之间的差异通常通过 $\phi$-散度来量化，这由一个熵函数 $\phi$ 唯一决定。

**定义 2.4** (熵函数)。熵函数 $\phi: \mathbb{R} \rightarrow \overline{\mathbb{R}}$ 是一个下半连续凸函数，满足 $\phi(1)=0$ 且 $\phi(s)=+\infty$ 对于所有 $s<0$。

注意，任何熵函数 $\phi$ 相对于其定义域是连续的。事实上，这对于任何一元的凸下半连续函数都是成立的。然而，我们强调，多元凸下半连续函数可能在其定义域内有不连续点（Rockafellar 和 Wets 2009, 示例 2.38）。$\phi$-散度的概念依赖于熵函数 $\phi$ 的视角 $\phi^\pi$。

**定义 2.5** ($\phi$-散度 (Csiszár 1963, 1967, Ali 和 Silvey 1966))。$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$ 相对于 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 的（广义）$\phi$-散度定义为

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})=\int_{\mathcal{Z}} \phi^\pi\left(\frac{\mathrm{d} \mathbb{P}}{\mathrm{~d} \rho}(z), \frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z)\right) \mathrm{d} \rho(z),
$$

其中 $\phi$ 是熵函数，$\rho \in \mathcal{M}_{+}(\mathcal{Z})$ 是任何支配测度。这意味着 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 相对于 $\rho$ 是绝对连续的，即 $\mathbb{P}, \hat{\mathbb{P}} \ll \rho$。

根据 $\phi^\pi$ 的定义以及我们的约定 $0 \phi(s / 0)$ 应该解释为 recession function $\phi^{\infty}(s)$，$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$ 可以重新表达为

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})=\int_{\mathcal{Z}} \frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z) \cdot \phi\left(\frac{\frac{\mathrm{d}}{\mathrm{P}}(z)}{\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z)}\right) \mathrm{d} \rho(z) .
$$

一个支配测度 $\rho$ 总是存在的，但它必须依赖于 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$。例如，可以设置 $\rho = \mathbb{P} + \hat{\mathbb{P}}$。绝对连续性条件 $\mathbb{P} \ll \rho$ 和 $\hat{\mathbb{P}} \ll \rho$ 确保了 Radon-Nikodym 导数 $\frac{\mathrm{d}}{\mathrm{d} \rho}$ 和 $\frac{\mathrm{d}}{\mathrm{d} \rho}$ 分别是良定义的。以下命题推导了一个通用的 $\phi$-散度的对偶表示，揭示了 $\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$ 实际上与选择 $\rho$ 无关。

命题 2.6 （$\phi$-散度的对偶表示）。我们有

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \, \mathrm{d}\mathbb{P}(z) - \int_{\mathcal{Z}} \phi^*(f(z)) \, \mathrm{d} \hat{\mathbb{P}}(z),
$$

其中 $\mathcal{F}$ 表示所有有界的 Borel 函数 $f: \mathcal{Z} \rightarrow \operatorname{dom}\left(\phi^*\right)$ 的集合。

Proof. As the entropy function $\phi(s)$ is proper, convex and lower semicontinuous on $\mathbb{R}$ and as $0 \phi(s / 0)$ is interpreted as the recession function $\phi^{\infty}(s)$, the perspective function $\phi^\pi(s, t)=t \phi(s / t)$ is proper, convex and lower semicontinuous on $\mathbb{R} \times \mathbb{R}_{+}$. By (Rockafellar 1970, Theorem 12.2), $\phi^\pi(s, t)$ can therefore be expressed as the conjugate of its conjugate. Note that the conjugate of $\phi^\pi(s, t)$ satisfies

$$
\begin{aligned}
\left(\phi^\pi\right)^*(f, g) & =\sup _{s \in \mathbb{R}, t \in \mathbb{R}_{+}} f s+g t-t \phi(s / t) \\
& =\sup _{t \in \mathbb{R}_{+}} g t+t \phi^*(f)=\left\{\begin{array}{cl}
0 & \text { if } f \in \operatorname{dom}\left(\phi^*\right) \text { and } g+\phi^*(f) \leq 0, \\
+\infty & \text { otherwise, }
\end{array}\right.
\end{aligned}
$$

for all $f, g \in \mathbb{R}$. The second equality in the above expression follows from (Rockafellar 1970, Theorem 16.1). As $\phi^\pi(s, t)=\sup _{f, g \in \mathbb{R}} s f+t g-\left(\phi^\pi\right)^*(f, g)$ by virtue of (Rockafellar 1970, Theorem 12.2), the $\phi$-divergence is thus given by

$$
\begin{aligned}
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) & =\int_{\mathcal{Z}} \sup _{f, g \in \mathbb{R}}\left\{\frac{\mathrm{~d} \mathbb{P}}{\mathrm{~d} \rho}(z) \cdot f+\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z) \cdot g-\left(\phi^\pi\right)^*(f, g)\right\} \mathrm{d} \rho(z) \\
& =\int_{\mathcal{Z}} \sup _{f \in \operatorname{dom}\left(\phi^*\right)}\left\{\frac{\mathrm{d} \mathbb{P}}{\mathrm{~d} \rho}(z) \cdot f-\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z) \cdot \phi^*(f)\right\} \mathrm{d} \rho(z) \\
& =\sup _{f \in \mathcal{F}} \int_{\mathcal{Z}}\left\{\frac{\mathrm{d} \mathbb{P}}{\mathrm{~d} \rho}(z) \cdot f(z)-\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z) \cdot \phi^*(f(z))\right\} \mathrm{d} \rho(z)
\end{aligned}
$$

第二个等式利用了我们上面推导出的$\left(\phi^\pi\right)^*$的显式公式，而第三个等式则根据（Rockafellar 和 Wets 2009, 定理 14.60）得出。这个定理之所以适用，是因为函数 $h: \operatorname{dom}\left(\phi^*\right) \times \mathcal{Z} \rightarrow \mathbb{R}$ 定义为

$$
h(f, z)=\frac{\mathrm{d} \mathbb{P}}{\mathrm{d} \rho}(z) \cdot f-\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{d} \rho}(z) \cdot \phi^*(f)
$$

在 $f$ 中是连续的，并且在 $z$ 中是 Borel 可测的，因此根据（Rockafellar 和 Wets 2009, 示例 14.29），它构成了一个 Carathéodory 积分器。然后，由于 Radon-Nikodym 导数的定义，该声明立即得出。

命题 2.6 显示了$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$在$\mathbb{P}$和$\hat{\mathbb{P}}$中是联合凸的。如果$\phi(s)$随着$s$的增长速度超过线性，即如果其渐近增长率$\phi^{\infty}(1)$是无穷大的，那么当且仅当$\frac{\mathrm{d}}{\mathrm{d} \rho}(z)=0$对于$\rho$几乎所有的$z \in \mathcal{Z}$，且$\frac{\mathrm{d} \hat{\mathrm{P}}}{\mathrm{d} \rho}(z)=0$时，$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$是有限的。换句话说，$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$只有在$\mathbb{P} \ll \hat{\mathbb{P}}$时才是有限的。在这种特殊情况下，Radon-Nikodym 导数的链式法则意味着$\frac{\mathrm{dP}}{\mathrm{d} \rho} / \frac{\mathrm{d} \mathrm{P}}{\mathrm{d} \rho}=\frac{\mathrm{dP}}{\mathrm{dP}}$。如果$\phi^{\infty}(1)=\infty$，则$\phi$-散度有更常见（但较不一般）的表示形式：

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})= \begin{cases}\int_{\mathcal{Z}} \phi\left(\frac{\mathrm{dP}}{\mathrm{~d} \hat{\mathbb{P}}}(z)\right) d \hat{\mathbb{P}}(z) & \text { if } \mathbb{P} \ll \hat{\mathbb{P}} \\ +\infty & \text { otherwise. }\end{cases}
$$

我们现在可以定义$\phi$-散度模糊集为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\} .
$$

该集合包含所有支持在$\mathcal{Z}$上的概率分布$\mathbb{P}$，其相对于某个指定参考分布$\hat{\mathbb{P}}$的$\phi$-散度最大为$r \geq 0$。


**Remark 2.7 (Csiszár 对偶性)**. 广义的 $\phi$-散度（即使 $\mathrm{P} \nless \hat{\mathrm{P}}$，也可以取有限值）在 P 和 $\hat{\mathrm{P}}$ 交换时是不变的。形式上，我们有 $\mathrm{D}_\phi(\mathbb{P}, \hat{P}) = \mathrm{D}_\phi(\hat{\mathbb{P}}, \mathbb{P})$，其中 $\psi$ 表示 $\phi$ 的 Csiszár 对偶，通过 $\psi(s) = \phi^\pi(1, s) = t \phi(1 / t)$ 定义（Ben-Tal, Ben-Israel 和 Teboulle 1991, 引理 2.3）。可以验证，如果 $\phi$ 是定义 2.4 中所定义的有效熵函数，那么 $\psi$ 也是一个有效熵函数。这一关系表明，尽管 $\phi$-散度通常是不对称的，但通过聚焦于形式为 (2.10) 的散度模糊集，其中名义分布 P 作为散度的第二个参数，我们并没有牺牲一般性。从 Proposition 2.6 之后的讨论中可以看出，如果 $\phi^{\infty}(1) = \infty$，那么所有在 $\phi$-散度模糊集 (2.10) 中的分布 $P$ 满足 $\mathbb{P} \ll \hat{\mathbb{P}}$。类似地，如果 $\phi$ 的 Csiszár 对偶满足 $\psi^{\infty}(1) = \infty$，那么所有在 $\phi$-散度模糊集中的分布 P 满足 $\hat{P} \ll \mathbb{P}$。

表 2.1 列出了常见的熵函数及其 Csiszár 对偶。我们强调，Cressie-Read 散度家族包括 Pearson $\chi^2$-散度（当 $\beta = 2$）、Kullback-Leibler 散度（当 $\beta \rightarrow 1$）以及似然散度（当 $\beta \rightarrow 0$）作为特例。

DRO 文献通常关注于受限的 $\phi$-散度模糊集

$$
\mathcal{P} = \left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{P} \ll \hat{\mathrm{P}}, \mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}
$$

该模糊集由 Ben-Tal 等人（2013）引入。与标准的 $\phi$-散度模糊集（2.10）不同，它仅包含相对于参考分布 $\hat{\mathbb{P}}$ 完全连续的分布，即使 $\phi^{\infty}(1) < \infty$。Ben-Tal 等人（2013）研究了在受限的 $\phi$-散度模糊集上的 DRO 问题，假设参考分布 $\hat{\mathbb{P}}$ 是离散的。在这种情况下，绝对连续性约束 $\mathbb{P} \ll \hat{\mathbb{P}}$ 确保模糊集仅包含支持在 $\hat{\mathbb{P}}$ 原子上的离散分布，因此自然的最坏情况期望问题转化为一个有限的凸规划问题。Ben-Tal 等人（2013）进一步发展了这一问题类的对偶性理论。Shapiro（2017）将这一对偶性理论扩展到一般的参考分布 $\hat{\mathbb{P}}$，这些参考分布不一定是离散的。Hu, Hong 和 So（2013）以及 Jiang 和 Guan（2016）表明，任何受限的 $\phi$-散度模糊集上的分布鲁棒单个机会约束都等价于参考分布 $\hat{\mathbb{P}}$ 下的经典机会约束，只是置信度水平进行了缩放。Bayraksan 和 Love（2015）提供了对各种 $\phi$-散度的分类，并分析了对应的 $\phi$-散度模糊集的结构性质，假设 $\mathcal{Z}$ 是有限的。下面我们回顾标准和受限的 $\phi$-散度模糊集的常见实例。



| Divergence                        | $\phi(s)~(s \geq 0)$                                  | $\psi(s)~(s \geq 0)$                                  | $\phi^\infty(1)$ | $\psi^\infty(1)$   |
|------------------------------------|-------------------------------------------------------|-------------------------------------------------------|------------------|---------------------|
| **Kullback-Leibler**               | $s \log(s) - s + 1$                                   | $- \log(s) + s - 1$                                   | $\infty$         | 1                   |
| **Likelihood**                     | $-\log(s) + s - 1$                                   | $s \log(s) - s + 1$                                   | 1                | $\infty$       |
| **Total variation**                | $\frac{1}{2}\|s - 1\|$  | $\frac{1}{2}\|s - 1\|$ | $\frac{1}{2}          $     | $\frac{1}{2}          $   |
| **Pearson $\chi^2$**               | $(s - 1)^2$                                           | $\frac{1}{s} (s - 1)^2$                               | $\infty$         | 1                   |
| **Neyman $\chi^2$**                | $\frac{1}{s} (s - 1)^2$                               | $(s - 1)^2$                                           | 1                | $\infty$           |
| **Cressie-Read for $\beta \in (0, 1)$** | $\frac{s^\beta - \beta s + \beta - 1}{\beta (\beta - 1)}$ | $s^\beta - \beta s + \beta - 1$                       | $\frac{1}{1 - \beta}$ | $\frac{1}{\beta}$ |
| **Cressie-Read for $\beta > 1$**   | $\frac{s^\beta - \beta s + \beta - 1}{\beta (\beta - 1)}$ | $s^\beta - \beta s + \beta - 1$                       | $\infty$         | $\frac{1}{\beta - 1}$ |


表 2.1. 熵函数及其 Csiszár 对偶的示例

### 2.2.1. Kullback-Leibler 模糊集

Kullback-Leibler 散度是对应于熵函数的 $\phi$-散度，该熵函数满足 $\phi(s) = s \log(s) - s + 1$，适用于所有 $s \geq 0$；详见表 2.1。由于 $\phi^{\infty}(1) = +\infty$，因此它具有以下等价定义。

**定义 2.8 (Kullback-Leibler 散度)**. 对于 $\mathbb{P} \in \mathcal{P}(\mathcal{Z})$ 和 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$，Kullback-Leibler 散度定义为

$$
\mathrm{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \begin{cases} 
\int_{\mathcal{Z}} \log \left(\frac{\mathrm{d} \mathbb{P}}{d \hat{\mathbb{P}}}(z)\right) \mathrm{d} \mathbb{P}(z) & \text{如果 } \mathbb{P} \ll \hat{\mathbb{P}}, \\
+\infty & \text{否则.}
\end{cases}
$$

我们现在回顾一下 Kullback-Leibler 散度的著名变分公式。

**命题 2.9 (Donsker 和 Varadhan (1983))**. 对于 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$，Kullback-Leibler 散度满足

$$
\operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \sup_{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z) - \log \left(\int_{\mathcal{Z}} e^{f(z)} \mathrm{d} \hat{\mathbb{P}}(z)\right),
$$

其中 $\mathcal{F}$ 表示所有有界的 Borel 函数 $f: \mathcal{Z} \rightarrow \mathbb{R}^d$ 的集合。

**证明**. 诱导 Kullback-Leibler 散度的熵函数 $\phi$ 的凸对偶满足 $\phi^*(t) = \exp(t) - 1$，其定义域为 $\mathbb{R}$。因此，命题 2.6 中建立的泛 $\phi$-散度的对偶表示式意味着

$$
\mathrm{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \sup_{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \mathrm{dP}(z) - \int_{\mathcal{Z}}\left(\mathrm{e}^{f(z)} - 1\right) \mathrm{d} \hat{\mathbb{P}}(z),
$$

其中 $\mathcal{F}$ 表示所有有界的 Borel 函数 $f: \mathcal{Z} \rightarrow \mathbb{R}$ 的集合。注意，$\mathcal{F}$ 在常数平移下是不变的。也就是说，如果 $f(z)$ 是有界的 Borel 函数，那么对于任意常数 $c \in \mathbb{R}$，$f(z) + c$ 也是有界的 Borel 函数。因此，在上述最大化问题中，我们可以在 $f \in \mathcal{F}$ 和 $c \in \mathbb{R}$ 上同时优化，从而得到

$$
\operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \sup_{f \in \mathcal{F}} \sup_{c \in \mathbb{R}} \int_{\mathcal{Z}}(f(z)+c) \mathrm{d\mathbb{P}}(z) - \int_{\mathcal{Z}}\left(\mathrm{e}^{f(z)+c}-1\right) \mathrm{d} \hat{\mathbb{P}}(z).
$$

对于任何固定的 $f \in \mathcal{F}$，关于 $c$ 的内部最大化问题的最优解是

$$
c^{\star} = -\log \left(\int_{\mathcal{Z}} e^{f(z)} \mathrm{d} \hat{\mathbb{P}}(z)\right).
$$

将这个表达式代入目标函数得到（2.12）。命题 2.9 建立了 Kullback-Leibler 散度与熵风险度量之间的联系。这个联系将在第 4.3 节中变得有用。

Kullback-Leibler 模糊集的半径为 $r \geq 0$，围绕 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 给出为

$$
\mathcal{P} = \{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\}.
$$

由于 $\phi^{\infty}(1) = +\infty$，因此所有的分布 $\mathbb{P} \in \mathcal{P}$ 都是相对于 $\hat{\mathbb{P}}$ 完全连续的。因此，$\mathcal{P}$ 与受限的 Kullback-Leibler 模糊集是相同的。El Ghaoui 等人（2003）推导了当 $\hat{\mathbb{P}}$ 是高斯分布时，线性损失函数的最坏情况value-at-risk 的封闭式表达式。Hu 和 Hong（2013）使用类似的方法表明，任何受限的 Kullback-Leibler 模糊集下的分布鲁棒个体机会约束都等价于参考分布 $\hat{\mathbb{P}}$ 下的经典机会约束，只是置信度水平被缩放了。Calafiore（2007）研究了当 $\hat{\mathbb{P}}$ 是离散分布时，最坏情况的均值-风险投资组合选择问题。Kullback-Leibler 模糊集还在最小二乘估计（Levy 和 Nikoukhah 2004）、假设检验（Levy 2008，Gül 和 Zoubir 2017）、滤波（Levy 和 Nikoukhah 2012，Zorzi 2016，2017a,b）、风险度量理论（Ahmadi-Javid 2012，Postek 等 2016）和极值分析（Blanchet, He 和 Murthy 2020）等方面找到了应用。


### 2.2.2. 似然模糊集

由于 Kullback-Leibler 散度并非对称，因此它产生了两个严格不同的模糊集。第二节中的 Kullback-Leibler 模糊集是通过将 Kullback-Leibler 散度的第二个参数固定为参考分布 $\hat{\mathbb{P}}$，并考虑所有满足 $\operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) \leq r$ 的分布 $\mathbb{P}$ 获得的。另一种模糊集则是通过使用 $\hat{\mathbb{P}}$ 作为第一个参数并设置

$$
\mathcal{P} = \{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{KL}(\hat{\mathbb{P}}, \mathbb{P}) \leq r\}.
$$

我们将 $\mathcal{P}$ 称为以 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 为中心的似然模糊集。事实上，$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$ 相对于 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 的似然或 Burg-熵散度通常定义为反向的 Kullback-Leibler 散度 $\operatorname{KL}(\hat{\mathbb{P}}, \mathbb{P})$。这种术语的基础是以下直觉。如果 $\mathcal{Z}$ 是一个离散集合，并且 $\hat{\mathbb{P}} = \frac{1}{N} \sum_{i=1}^N \delta_{\hat{z}_i}$ 是与来自未知分布的 $N$ 个独立样本 $\left\{\hat{z}_i\right\}_{i=1}^N$ 相对应的经验分布，那么构造所有分布族，使得观察到的数据在规定的似然水平下能够实现，便是自然的。这些分布族对应于似然函数 $\mathcal{L}(\mathbb{P})=\prod_{i=1}^N \mathbb{P}\left(Z=\hat{z}_i\right)$ 在 $\mathcal{P}(\mathcal{Z})$ 上的超水平集。可以证明，任何这样的超水平集与似然散度 $\operatorname{KL}(\hat{\mathbb{P}}, \mathbb{P})$ 的子水平集一致。因此，它构成了形式为 (2.14) 的似然模糊集。我们强调，这种对应关系不容易延伸到 $\mathcal{Z}$ 不是离散的情况。

似然模糊集最初由 Wang 等人（2016）在静态 DRO 的背景下提出，后来被 Wiesemann、Kuhn 和 Rustem（2013）用于稳健的马尔可夫决策过程。Bertsimas 等人（2018a,b）表明，似然模糊集包含所有在规定显著性水平下通过 G 检验的分布。

即使 $\mathcal{Z}$ 是不可数的，似然模糊集也显示出几个统计最优性性质。为了说明这些性质，我们考虑以下任务：在给定来自 $\mathbb{P}$ 的 $N$ 个独立样本的情况下，评估某个损失函数在未知分布 $\mathbb{P}$ 下的 $(1-\eta)$ 上置信区间。利用 Owen（1988）提出的经验似然定理，Lam（2019）展示了以经验分布 $\hat{\mathbb{P}}$ 为中心的似然模糊集的一个理想性质：与之关联的最坏情况期望损失提供了对常数显著性水平 $\eta$ 的最不保守的置信区间，当半径 $r$ 以 $1 / N$ 的速度衰减时，渐近地得出类似保证。Duchi、Glynn 和 Namkoong（2021）报告了对更广泛类别的 $\phi$-散度的类似保证。此外，Van Parys、Mohajerin Esfahani 和 Kuhn（2021）利用 Sanov 的大偏差原理（Cover 和 Thomas 2006，第 11.4.1 定理）证明，针对以常数半径 $r$ 为中心的似然模糊集，最坏情况的期望损失为渐近下显著性水平 $\eta \propto e^{-r N}$ 的最不保守置信区间。Gupta（2019）进一步表明，围绕 $\hat{\mathbb{P}}$ 的半径 $r \propto N^{-1/2}$ 的似然模糊集代表了满足贝叶斯鲁棒性保证的最小凸模糊集。


### 2.2.3. 总变差模糊集

两个分布 $\mathbb{P}, \hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 的总变差距离是 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 在任何事件上所分配的概率之间的最大绝对差异。

**定义 2.10（总变差距离）** 总变差距离是定义为

$$
\operatorname{TV}(\mathbb{P}, \hat{\mathbb{P}}) = \sup \{|\mathbb{P}(\mathcal{B}) - \hat{\mathbb{P}}(\mathcal{B})| : \mathcal{B} \subseteq \mathcal{Z} \text{ 是一个 Borels 集合} \}
$$

总变差距离显然是对称的，满足不可区分的恒等式和三角不等式。因此，它构成了 $\mathcal{P}(\mathcal{Z})$ 上的一个度量。此外，总变差距离是一个 $\phi$-散度的实例。

** 命题 2.11.** 总变差距离与由熵函数 $\phi(s) = \frac{1}{2}|s - 1|$ 对所有 $s \geq 0$ 引起的 $\phi$-散度一致。

**证明：** 诱发 Kullback-Leibler 散度的熵函数的共轭熵函数满足 $\phi^*(t) = \max\{t, -\frac{1}{2}\}$，当 $t \leq \frac{1}{2}$ 时，$\phi^*(t) = +\infty$ 当 $t > \frac{1}{2}$ 时。根据命题 2.6，与给定熵函数对应的 $\phi$-散度因此具有如下的对偶表示：

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) = \sup _{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z) - \int_{\mathcal{Z}} \max\{f(z), -\frac{1}{2}\} \mathrm{d} \hat{\mathbb{P}}(z),
$$

其中 $\mathcal{F}$ 表示所有有界的 Borel 函数 $f: \mathcal{Z} \rightarrow \left(-\infty, \frac{1}{2}\right]$ 的族。注意，$\mathcal{F}$ 对常数平移不变。也就是说，如果 $f(z)$ 是一个有界的 Borel 函数，那么 $f(z) + c$ 对于任何常数 $c \in \mathbb{R}$ 也是有界的 Borel 函数。因此，我们可以在上述最大化问题中同时对 $f \in \mathcal{F}$ 和 $c \in \mathbb{R}$ 进行优化，从而得到：

$$
\operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \sup _{f \in \mathcal{F}} \sup _{c \in \mathbb{R}} \int_{\mathcal{Z}} (f(z) + c) \mathrm{d\mathbb{P}}(z) - \int_{\mathcal{Z}} \left(\mathrm{e}^{f(z) + c} - 1\right) \mathrm{d} \hat{\mathbb{P}}(z).
$$

对于任何固定的 $f \in \mathcal{F}$，关于 $c$ 的内层最大化问题可以通过以下方式唯一求解：

$$
c^{\star} = -\log \left(\int_{\mathcal{Z}} \mathrm{e}^{f(z)} \mathrm{d} \hat{\mathbb{P}}(z)\right).
$$

将此表达式代回目标函数中，得到（2.12）。命题 2.9 建立了 Kullback-Leibler 散度与熵风险度量之间的联系。这个联系将在第 4.3 节中变得有用。

Kullback-Leibler 模糊集的半径为 $r \geq 0$ 的模糊集定义为：

$$
\mathcal{P} = \{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\}.
$$

由于 $\phi^{\infty}(1) = +\infty$，因此所有分布 $\mathbb{P} \in \mathcal{P}$ 都是相对于 $\hat{\mathbb{P}}$ 的绝对连续的。因此，$\mathcal{P}$ 与限制 Kullback-Leibler 模糊集一致。El Ghaoui 等人（2003）推导出了当 $\hat{\mathbb{P}}$ 是高斯分布时，线性损失函数的最坏情况价值风险的封闭形式表达式。Hu 和 Hong（2013）使用类似的技术表明，任何相对于 Kullback-Leibler 模糊集的分布鲁棒性个体机会约束等价于参考分布 $\hat{\mathbb{P}}$ 下的经典机会约束，但具有重新调整的置信水平。Calafiore（2007）研究了当 $\hat{\mathbb{P}}$ 是离散分布时，最坏情况下的均值风险投资组合选择问题。Kullback-Leibler 模糊集还在最小二乘估计（Levy 和 Nikoukhah 2004）、假设检验（Levy 2008, Gül 和 Zoubir 2017）、滤波（Levy 和 Nikoukhah 2012, Zorzi 2016, 2017a,b）、风险度量理论（Ahmadi-Javid 2012, Postek 等人 2016）和极值分析（Blanchet, He 和 Murthy 2020）等领域找到了应用。

半径为 $r \geq 0$ 的总变差模糊集围绕 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 定义为：

$$
\mathcal{P}=\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{TV}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\}
$$

现有文献大多集中在限制性总变差模糊集上，该模糊集包含所有满足 $\mathbb{P} \ll \hat{\mathbb{P}}$ 的分布 $\mathbb{P} \in \mathcal{P}$。Jiang 和 Guan (2018, Theorem 1) 以及 Shapiro (2017, Example 3.7) 证明，相对于限制性总变差模糊集的最坏期望损失与条件价值-at-risk 和相对于 $\hat{\mathbb{P}}$ 的损失的本质上确界的组合一致，详见第6.10节。Rahimian、Bayraksan 和 Homem-deMello (2019a, $b$, 2022) 研究了在 $\mathcal{Z}$ 是有限集的情况下，未限制总变差模糊集上的 DRO 问题的最坏分布。总变差模糊集与鲁棒统计中的 Huber 污染模型相关（Huber 1981），该模型假设统计数据集中的一部分样本（比例为 $r \in(0,1)$）来自于一个任意的污染分布。因此，目标分布和污染数据生成分布之间的总变差距离至多为 $r$。因此，围绕某个估计分布的半径 $r$ 的总变差模糊集作为目标分布的搜索空间是自然的选择（Nishimura 和 Ozaki 2004, 2006；Bose 和 Daripa 2009；Duchi, Hashimoto 和 Namkoong 2023；Tsanga 和 Shehadeha 2024）。


### 2.2.4. $\chi^2$-Divergence 模糊集

$\chi^2$-散度是对应于熵函数的 $\phi$-散度，该熵函数满足对于所有 $s \geq 0$，有 $\phi(s)=(s-1)^2$；另见表 2.1。由于 $\phi^{\infty}(1)=+\infty$，因此它有以下等价定义。

定义 2.12 ( $\chi^2$-散度)。对于 $\mathbb{P} \in \mathcal{P}(\mathcal{Z})$，相对于 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 的 $\chi^2$-散度定义为

$$
\chi^2(\mathbb{P}, \hat{\mathbb{P}})= \begin{cases}\int_{\mathcal{Z}}\left(\frac{\mathrm{d} \mathbb{P}}{\mathrm{~d} \hat{\mathbb{P}}}(z)-1\right)^2 \mathrm{~d} \hat{\mathbb{P}}(z) & \text { 如果 } \mathbb{P} \ll \hat{\mathbb{P}} \\ +\infty & \text { 否则 }\end{cases}
$$

$\chi^2$-散度有以下对偶表示。

**命题 2.13**.对于 $\mathbb{P}$，相对于 $\hat{\mathbb{P}}$ 的 $\chi^2$-散度满足

$$
\chi^2(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{F}} \frac{\left(\mathbb{E}_{\mathbb{P}}[f(Z)]-\mathbb{E}_{\hat{\mathbb{p}}}[f(Z)]\right)^2}{\mathbb{V}_{\hat{\mathbb{P}}}[f(Z)]},
$$

其中 $\mathcal{F}$ 表示所有有界 Borel 函数的集合 $f: \mathcal{Z} \rightarrow \mathbb{R}$，$\mathbb{V}_{\hat{\mathbb{P}}}[f(Z)]$ 表示 $f(Z)$ 在 $\hat{\mathbb{P}}$ 下的方差。如果 $\mathbb{V}_{\hat{\mathbb{P}}}[f(Z)] = 0$，则上述分数被解释为 0（如果 $\mathbb{E}_{\mathbb{P}}[f(Z)] = \mathbb{E}_{\hat{\mathbb{P}}}[f(Z)]$）或 $+\infty$（否则）。

证明。引起 $\chi^2$-散度的熵函数的凸共轭满足 $\phi^*(t)=\frac{t^2}{4}+t$ 如果 $t \geq-2$，并且如果 $t<-2$，则 $\phi^*(t)=-1$，其定义域为 $\operatorname{dom}\left(\phi^*\right)=\mathbb{R}$。因此，命题 2.6 表明

$$
\chi^2(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}}\left(\frac{f(z)^2}{4}+f(z)\right) \mathrm{d} \hat{\mathbb{P}}(z),
$$

其中 $\mathcal{F}$ 表示所有有界 Borel 函数 $f: \mathcal{Z} \rightarrow \mathbb{R}$。请注意，我们在第二个积分中将 $\phi^*(f(z))$ 替换为 $f(z)^2 / 4 + f(z)$。这样做没有损失一般性。事实上，如果函数 $f(z)$ 取值小于 -2，那么它（弱）被函数 $f^{\prime}(z)=\max \{f(z),-2\}$ 所主导。还要注意，$\mathcal{F}$ 对常数偏移不变。也就是说，如果 $f(z)$ 是有界的 Borel 函数，那么 $f(z)+c$ 对于任何常数 $c \in \mathbb{R}$ 也是有界的。一个简单的计算表明，对于任何固定的 $f \in \mathcal{F}$，最优的偏移量是 $c^{\star}=-\mathbb{E}_{\hat{\mathbb{p}}}[f(Z)]$。因此，我们可以将 $f(z)$ 替换为 $f(z)-\mathbb{E}_{\hat{\mathbb{p}}}[f(Z)]$，这将得到

$$
\chi^2(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{F}} \mathbb{E}_{\mathbb{P}}[f(Z)]-\mathbb{E}_{\hat{\mathbb{P}}}[f(Z)]-\frac{\mathbb{V}_{\hat{\mathbb{P}}}[f(Z)]}{4} .
$$

注意，集合 $\mathcal{F}$ 对缩放也不变。也就是说，如果 $f(z)$ 是有界的 Borel 函数，那么 $c f(z)$ 对于任何常数 $c \in \mathbb{R}$ 也是有界的。因此，我们可以分别在上述最大化问题中对 $f \in \mathcal{F}$ 和 $c \in \mathbb{R}$ 进行优化，从而得到

$$
\begin{aligned}
\chi^2(\mathbb{P}, \hat{\mathbb{P}}) & =\sup _{f \in \mathcal{F}} \sup _{c \in \mathbb{R}}\left(\mathbb{E}_{\mathbb{P}}[f(Z)]-\mathbb{E}_{\hat{\mathbb{P}}}[f(Z)]\right) c-\frac{\mathbb{V}_{\hat{\mathbb{P}}}[f(Z)]}{4} c^2 \\
& =\sup _{f \in \mathcal{F}} \frac{\left(\mathbb{E}_{\mathbb{P}}[f(Z)]-\mathbb{E}_{\hat{\mathbb{P}}}[f(Z)]\right)^2}{\mathbb{V}_{\hat{\mathbb{P}}}[f(Z)]} .
\end{aligned}
$$

注意，内层的最大化问题对 $c$ 进行求解时，实际上是在 $\mathbb{V}_{\hat{\mathbb{P}}}[f(Z)] c^2 / 4$ 上计算共轭函数，这可以用闭式形式得出。因此，结论得以成立。

由于 $\chi^2$-散度不是对称的，它产生了两个互补的模糊集，这取决于参考分布 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 是作为 $\chi^2$-散度的第一个还是第二个参数来使用。Lam (2018) 定义了围绕 $\hat{\mathbb{P}}$ 的 Pearson $\chi^2$-模糊集，半径为 $r \geq 0$：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \chi^2(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}
$$

用于分析具有依赖数据的操作和服务系统。Philpott、de Matos 和 Kapelevich (2018) 发展了一种随机对偶动态规划算法，用于求解具有 Pearson 模糊集的分布鲁棒多阶段随机程序。在静态 DRO 的背景下，Duchi 和 Namkoong (2019) 证明了，相对于 Pearson 模糊集的鲁棒化与方差正则化密切相关。请注意，由于 $\phi^{\infty}(1)=+\infty$，Pearson 模糊集与其限制版本相同，后者仅包含满足 $\mathbb{P} \ll \hat{\mathbb{P}}$ 的分布。

Klabjan、Simchi-Levi 和 Song (2013) 定义了 Neyman $\chi^2$-模糊集：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \chi^2(\hat{\mathbb{P}}, \mathbb{P}) \leq r\right\}
$$

用于表述鲁棒批量选择问题。Hanasusanto 和 Kuhn (2013) 在有限的 $\mathcal{Z}$ 上使用了 Neyman 模糊集，背景是鲁棒数据驱动的动态规划。最后，Hanasusanto 等人 (2015a) 使用相同的模糊集来建模多模态需求分布的混合权重的不确定性。


2.3. 最优运输模糊集

最优运输理论为量化概率分布之间的差异提供了一种自然的方式，并产生了一类丰富的模糊集。为了说明这一点，我们首先引入运输成本函数的概念。

定义 2.14 (运输成本函数)。一个下半连续函数 $c: \mathcal{Z} \times \mathcal{Z} \rightarrow[0,+\infty]$，满足对于所有 $z \in \mathcal{Z}$，有 $c(z, z)=0$，称为运输成本函数。

每个运输成本函数都引发一个最优运输差异。

定义 2.15 (最优运输差异)。与给定的运输成本函数 $c$ 相关的最优运输差异 $\mathrm{OT}_c: \mathcal{P}(\mathcal{Z}) \times \mathcal{P}(\mathcal{Z}) \rightarrow[0,+\infty]$ 定义为：

$$
\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})=\inf _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})} \mathbb{E}_\gamma[c(Z, \hat{Z})],
$$

其中 $\Gamma(\mathbb{P}, \hat{\mathbb{P}})$ 表示 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 的所有耦合 $\gamma$ 的集合，也就是所有具有边际 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 的 $Z$ 和 $\hat{Z}$ 的联合概率分布。

根据定义，当且仅当 $\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})$ 时，我们有 $\gamma((Z, \hat{Z}) \in \mathcal{B} \times \mathcal{Z})=\mathbb{P}(Z \in \mathcal{B})$ 和 $\gamma((Z, \hat{Z}) \in \mathcal{Z} \times \hat{\mathcal{B}})=\hat{\mathbb{P}}(\hat{Z} \in \mathcal{B})$ 对所有 Borel 集 $\mathcal{B}$，$\mathcal{B} \subseteq \mathcal{Z}$。若把概率分布 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 看作两堆沙子，那么任何耦合 $\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})$ 可以被解释为一个运输计划，也就是通过在 $\mathcal{Z}$ 中移动沙子来将 $\hat{\mathbb{P}}$ 变形为 $\mathbb{P}$。事实上，对于任何固定的源点 $\hat{z} \in \mathcal{Z}$，条件概率 $\gamma(z \leq Z \leq z+\mathrm{d} z \mid \hat{Z}=\hat{z})$ 确定了从 $\hat{z}$ 移动到目标点 $z$ 的沙子的比例。如果将一个单位的概率质量从 $\hat{z}$ 移动到 $z$ 的成本为 $c(z, \hat{z})$，那么 $\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})$ 就是将 $\hat{\mathbb{P}}$ 变形为 $\mathbb{P}$ 所需的最小费用。接下来我们为一般的最优运输差异提供一个对偶表示。

**命题 2.16** (Kantorovich 对偶性 I)。我们有：

$$
\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})=\left\{\begin{array}{cl}
\sup _{f \in \mathcal{L}^1(\mathbb{P}), g \in \mathcal{L}^1(\hat{\mathbb{P}})} & \int_{\mathcal{Z}} f(z) \mathrm{dP}(z)-\int_{\mathcal{Z}} g(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) \\
\text { s.t. } & f(z)-g(\hat{z}) \leq c(z, \hat{z}) \quad \forall z, \hat{z} \in \mathcal{Z},
\end{array}\right.
$$

其中 $\mathcal{L}^1(\mathbb{P})$ 和 $\mathcal{L}^1(\hat{\mathbb{P}})$ 表示从 $\mathcal{Z}$ 到 $\mathbb{R}$ 的所有有界 Borel 函数集合，这些函数分别是相对于 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 可积的。

对偶问题 (2.19) 代表了第三方的利润最大化问题，第三方代表问题所有者将沙子从 $\hat{\mathbb{P}}$ 分配到 $\mathbb{P}$，通过在源点 $\hat{z}$ 以单位价格 $g(\hat{z})$ 买入沙子，并在目标点 $z$ 以单位价格 $f(z)$ 卖出沙子。约束条件确保，对于每个源点-目标点对 $(\hat{z}, z)$，对问题所有者来说，使用第三方服务比在运输成本 $c(z, \hat{z})$ 下自己移动沙子要便宜。最优价格函数 $f^{\star}$ 和 $g^{\star}$（如果存在的话）被称为 Kantorovich 潜力。

命题 2.16 证明。关于一般证明，请参见（Villani 2008，定理 5.10（i））。我们在简化假设 $\mathcal{Z}$ 是紧集的情况下证明这一主张。在这种情况下，所有连续（因此有界）的函数 $f: \mathcal{Z} \times \mathcal{Z} \rightarrow \mathbb{R}$ 所组成的家族 $\mathcal{C}(\mathcal{Z} \times \mathcal{Z})$，配备上极大值范数，构成了一个 Banach 空间。它的拓扑对偶空间是所有有限有符号 Borel 测度的空间 $\mathcal{M}(\mathcal{Z} \times \mathcal{Z})$，配备总变差范数（Folland 1999，推论 7.18）。这意味着，对于每一个连续线性泛函 $\varphi: \mathcal{C}(\mathcal{Z} \times \mathcal{Z}) \rightarrow \mathbb{R}$，都存在 $\gamma \in \mathcal{M}(\mathcal{Z} \times \mathcal{Z})$，使得对于所有 $f \in \mathcal{C}(\mathcal{Z} \times \mathcal{Z})$，有 $\varphi(f)=\int_{\mathcal{Z} \times \mathcal{Z}} f(z, \hat{z}) \mathrm{d} \gamma(z, \hat{z})$。

我们首先使用 Fenchel - Rockafellar 对偶定理来证明：

$$
\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})=\left\{\begin{array}{cl}
\sup _{f, g \in \mathcal{C}(\mathcal{Z})} & \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} g(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) \\
\text { s.t. } & f(z)-g(\hat{z}) \leq c(z, \hat{z}) \quad \forall z, \hat{z} \in \mathcal{Z},
\end{array}\right.
$$

即我们证明了如果对偶问题中的价格函数 $f$ 和 $g$ 被限制在 $\mathcal{C}(\mathcal{Z})$ 空间中，强对偶性成立。为此，我们将（2.20）中的最大化问题重新表达为更加简洁的形式：

$$
\sup _{h \in \mathcal{C}(\mathcal{Z} \times \mathcal{Z})}-\phi(h)-\psi(h),
$$

其中凸函数 $\phi, \psi: \mathcal{C}(\mathcal{Z} \times \mathcal{Z}) \rightarrow(-\infty,+\infty]$ 通过以下方式定义：

$$
\phi(h)= \begin{cases}0 & \text { 如果 }-h(z, \hat{z}) \leq c(z, \hat{z}) \forall z, \hat{z} \in \mathcal{Z}, \\ +\infty & \text { 否则, }\end{cases}
$$

和

$$
\psi(h)= \begin{cases}\int_{\mathcal{Z}} \int_{\mathcal{Z}} h(z, \hat{z}) \mathrm{d} \mathbb{P}(z) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) & \left\{\begin{array}{l}
\text { 如果 } \exists f, g \in \mathcal{C}(\mathcal{Z}) \text { 且 } \\ 
\\
h(z, \hat{z})=g(\hat{z})-f(z) \forall z, \hat{z} \in \mathcal{Z}, \\
+\infty
\end{array}\right. \\
\text { 否则. }\end{cases}
$$

注意，(2.21) 可以看作是相对于 $\mathcal{C}(\mathcal{Z} \times \mathcal{Z})$ 和 $\mathcal{M}(\mathcal{Z} \times \mathcal{Z})$ 的配对在零测度处的对偶。注意到，由于运输成本函数 $c$ 是非负的，$\phi$ 在常数函数 $h_0 \equiv 1$ 处是连续的。此外，$h_0$ 属于 $\psi$ 的定义域。因此，Fenchel-Rockafellar 对偶定理（Brezis 2011，定理 1.12）确保了凸函数 $\phi$ 和 $\psi$ 的和的对偶与它们的对偶函数 $\phi^*$ 和 $\psi^*$ 的极小卷积一致。因此，(2.21) 等于：

$$
(\phi+\psi)^*(0)=\inf _{\gamma \in \mathcal{M}(\mathcal{Z} \times \mathcal{Z})} \phi^*(-\gamma)+\psi^*(\gamma) .
$$

接下来，我们需要评估 $\phi$ 和 $\psi$ 的对偶。对于任意 $\gamma \in \mathcal{M}(\mathcal{Z} \times \mathcal{Z})$，我们有：

$$
\begin{aligned}
\phi^*(-\gamma) & =\sup _{h \in \mathcal{C}(\mathcal{Z} \times \mathcal{Z})}\left\{-\int_{\mathcal{Z} \times \mathcal{Z}} h(z, \hat{z}) \mathrm{d} \gamma(z, \hat{z}): -h(z, \hat{z}) \leq c(z, \hat{z}) \forall z, \hat{z} \in \mathcal{Z}\right\} \\
& = \begin{cases}\int_{\mathcal{Z} \times \mathcal{Z}} c(z, \hat{z}) \mathrm{d} \gamma(z, \hat{z}) & \text { 如果 } \gamma \in \mathcal{M}_{+}(\mathcal{Z} \times \mathcal{Z}) \\
+\infty & \text { 否则 }\end{cases}
\end{aligned}
$$

其中 $\mathcal{M}_{+}(\mathcal{Z} \times \mathcal{Z})$ 代表 $\mathcal{Z} \times \mathcal{Z}$ 上的有限 Borel 测度的锥体。实际上，如果 $\gamma \in \mathcal{M}_{+}(\mathcal{Z} \times \mathcal{Z})$，那么第二个等式来自单调收敛定理，因其应用于 $c$ 是下半连续的，因此可以写成一列非递减的连续函数的逐点极限（参见下面的引理 3.1）。另一方面，如果 $\gamma \notin \mathcal{M}_{+}(\mathcal{Z} \times \mathcal{Z})$，那么第二个等式成立，因为每个 $\gamma \in \mathcal{M}(\mathcal{Z} \times \mathcal{Z})$ 都是 Radon 测度，这保证了任何 Borel 集合的测度可以通过连续函数的积分来逼近。类似地，对于任何 $\gamma \in \mathcal{M}(\mathcal{Z} \times \mathcal{Z})$，我们可以立即验证：

$$
\psi^*(\gamma)= \begin{cases}0 & \text { 如果 } \gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}}) \\ +\infty & \text { 否则. }\end{cases}
$$

将上述 $\phi^*$ 和 $\psi^*$ 的公式代入（2.22）中即可得到（2.20）。
放宽对偶问题右侧对 $f, g \in \mathcal{C}(\mathcal{Z})$ 的要求为 $f \in \mathcal{L}^1(\mathbb{P})$ 和 $g \in \mathcal{L}^1(\hat{\mathbb{P}})$，立即得到上界：

$$
\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}}) \leq\left\{\begin{array}{cl}
\sup _{f \in \mathcal{L}^1(\mathrm{P}), g \in \mathcal{L}^1(\hat{\mathbb{P}})} & \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} g(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) \\
\text { s.t. } & f(z)-g(\hat{z}) \leq c(z, \hat{z}) \quad \forall z, \hat{z} \in \mathcal{Z}
\end{array}\right.
$$

另一方面，很明显：

$$
\begin{aligned}
\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})= & \inf _{\gamma \in \mathcal{M}_{+}(\mathcal{Z} \times \mathcal{Z})} \sup _{f \in \mathcal{L}^1(\mathbb{P}), g \in \mathcal{L}^1(\hat{\mathbb{P}})} \int_{\mathcal{Z} \times \mathcal{Z}}(c(z, \hat{z})-f(z)+g(\hat{z})) \mathrm{d} \gamma(z, \hat{z}) \\
& +\int_{\mathcal{Z}} f(z) \mathrm{dP}(z)-\int_{\mathcal{Z}} g(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) .
\end{aligned}
$$

交换上述表达式中的最小化和最大化顺序，然后闭式计算内部的最小值，得到：

$$
\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}}) \geq\left\{\begin{array}{cl}
\sup _{f \in \mathcal{L}^1(\mathbb{P}), g \in \mathcal{L}^1(\hat{\mathbb{P}})} & \int_{\mathcal{Z}} f(z) \mathrm{dP}(z)-\int_{\mathcal{Z}} g(\hat{z}) \mathrm{d}(\hat{\mathbb{P}}(\hat{z}) \\
\text { s.t. } & f(z)-g(\hat{z}) \leq c(z, \hat{z}) \quad \forall z, \hat{z} \in \mathcal{Z} .
\end{array}\right.
$$

结合（2.23）和（2.24）证明了（2.19），因此结论成立。


## 2.3. 最优运输模糊集

最优运输问题（2.19）构成了一个关于价格函数 $f \in \mathcal{L}^1(\mathbb{P})$ 和 $g \in \mathcal{L}^1(\hat{\mathbb{P}})$ 的线性规划，其目标函数在 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 上是线性的。由于线性函数的逐点上确界是凸的，因此 $\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})$ 在 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 上是联合凸的。问题（2.19）可以通过引入价格函数 $f$ 的 $c$-变换 $f^c: \mathcal{Z} \rightarrow(-\infty,+\infty]$ 进一步简化，$f^c$ 定义为：

$$
f^c(\hat{z})=\sup _{z \in \mathcal{Z}} f(z)-c(z, \hat{z}) .
$$

现在，双对偶问题（2.19）的约束条件可以重新表示为：

$$
g(\hat{z}) \geq f(z)-c(z, \hat{z}) \quad \forall z, \hat{z} \in \mathcal{Z} \quad \Longleftrightarrow \quad g(\hat{z}) \geq f^c(\hat{z}) \quad \forall \hat{z} \in \mathcal{Z} .
$$

请注意，问题（2.19）寻求一个尽可能小的价格函数 $g$。由于 $g$ 被 $f^c$ 下界，因此这表明在最优解时 $g=f^c$。反之，通过定义价格函数 $g$ 的 $c$-变换 $g^c: \mathcal{Z} \rightarrow[-\infty,+\infty)$，通过以下方式定义：

$$
g^c(z)=\inf _{\hat{z} \in \mathcal{Z}} g(\hat{z})+c(z, \hat{z})
$$

问题（2.19）的约束条件可以重新表示为：

$$
f(z) \leq g(\hat{z})+c(z, \hat{z}) \quad \forall z, \hat{z} \in \mathcal{Z} \quad \Longleftrightarrow \quad f(z) \leq g^c(z) \quad \forall z \in \mathcal{Z} .
$$

这表明在最优解时 $f=g^c$。请注意，$f^c$ 和 $g^c$ 是上半连续函数的逐点上确界，因此它们也是上半连续的。此外，注意到 $f^c$ 和 $g^c$ 可能无法相对于 $\hat{\mathbb{P}}$ 和 $\mathbb{P}$ 分别可积。然而，如果 $f \in \mathcal{L}^1(\mathbb{P})$ 且 $g \in \mathcal{L}^1(\hat{\mathbb{P}})$，则可以验证积分 $\int_{\mathcal{Z}} f^c(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z})<+\infty$ 和 $\int_{\mathcal{Z}} g^c(z) \mathrm{dP}(z)>-\infty$ 存在为扩展实数。上述洞察最终得出以下推论，我们不提供证明。有关详细信息，请参见（Villani 2008，定理 5.10（i））。

**推论 2.17** (Kantorovich 对偶性 II)。我们有：

$$
\begin{aligned}
\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}}) & =\sup _{f \in \mathcal{L}^1(\mathbb{P})} \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} f^c(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) \\
& =\sup _{g \in \mathcal{L}^1(\hat{\mathbb{P}})} \int_{\mathcal{Z}} g^c(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} g(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}),
\end{aligned}
$$

其中 $f^c$ 和 $g^c$ 的 $c$-变换分别在（2.25）和（2.26）中定义。此外，如果我们要求对于某个函数 $g: \mathcal{Z} \rightarrow(-\infty,+\infty](f: \mathcal{Z} \rightarrow[-\infty,+\infty))$，则第一个（第二个）上确界不变，要求 $f=g^c$ $\left(g=f^c\right)$。

给定任何运输成本函数 $c$，参考分布 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 和运输预算 $r \geq 0$，最优运输模糊集定义为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\} .
$$

通过构造，$\mathcal{P}$ 包含所有能够通过在最大成本 $r \geq 0$ 内变形参考分布 $\hat{\mathbb{P}}$ 获得的概率分布 $\mathbb{P}$。最优运输模糊集最早由 Pflug 和 Wozabal（2007）研究，他们提出了一种连续线性规划算法，用于解决当 $\mathcal{Z}$ 有限时的稳健均值风险投资组合选择问题。Postek 等人（2016）利用对偶理论工具，开发了一种精确解法来解决相同问题类。Wozabal（2012）和 Pflug 和 Pichler（2014，§ 7.1）将具有最优运输模糊集的 DRO 问题重新表述为在 $\mathcal{Z} \subseteq \mathbb{R}^d$ 上的无穷支持集的有限维非凸问题，并采用全局优化方法解决它们。Mohajerin Esfahani 和 Kuhn（2018）以及 Zhao 和 Guan（2018）使用专门的对偶性结果，表明这些 DRO 问题实际上等价于广义的矩问题，并且可以准确地重新表述为有限维凸程序。Blanchet 和 Murthy（2019），Gao 和 Kleywegt（2023），以及 Zhang 等人（2024b）表明，即使 $\mathcal{Z}$ 是一个 Polish 空间，基础对偶性结果仍然有效。有关具有最优运输模糊集的 DRO 理论和应用的最新综述，请参阅 Kuhn 等人（2019）和 Blanchet、Murthy 和 Nguyen（2021）。


### 2.3.1. $p$-Wasserstein模糊集

在定义2.15中，通常将运输成本函数 $c$ 设置为某个度量 $d(\cdot, \cdot)$ 的 $p$ 次方。在这种情况下，最优运输差异的 $p$ 次方根被称为 $p$-Wasserstein 距离。

定义 2.18 ($p$-Wasserstein 距离)。假设 $d(\cdot, \cdot)$ 是 $\mathcal{Z}$ 上的度量，$p \in[1,+\infty)$ 是给定的指数。那么，$p$-Wasserstein 距离 $\mathrm{W}_p: \mathcal{P}(\mathcal{Z}) \times \mathcal{P}(\mathcal{Z}) \rightarrow[0,+\infty]$ 对应于 $d$ 和 $p$ 通过以下方式定义：

$$
\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})=\inf _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})}\left(\mathbb{E}_\gamma\left[d(Z, \hat{Z})^p\right]\right)^{\frac{1}{p}} .
$$

**定义 2.18** 意味着如果 $c(z, \hat{z})=d(z, \hat{z})^p$，则有 $W_p^p(\mathbb{P}, \hat{\mathbb{P}})=\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})$。在以下内容中，我们用 $\mathcal{P}_p(\mathcal{Z})=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}\left[d\left(Z, \hat{z}_0\right)^p\right]<\infty\right\}$ 来表示所有在 $\mathcal{Z}$ 上具有有限 $p$-阶矩的分布的集合。由于 $d$ 是度量，$\mathcal{P}_p(\mathcal{Z})$ 与参考点 $\hat{z}_0 \in \mathcal{Z}$ 的选择无关。$p$-Wasserstein 距离构成了 $\mathcal{P}_p(\mathcal{Z})$ 上的度量。事实上，很明显，$W_p(\mathbb{P}, \hat{\mathbb{P}})$ 是对称的，并且当且仅当 $\mathbb{P}=\hat{\mathbb{P}}$ 时为零。证明 $W_p(\mathbb{P}, \hat{\mathbb{P}})$ 满足三角不等式需要一个关于运输计划的粘合引理，因此更为复杂；参见（Villani 2008，第1章）。$p$-Wasserstein 距离进一步使得分布的弱收敛和它们的 $p$-阶矩的收敛成为度量。这意味着 $W_p\left(\mathbb{P}, \hat{\mathbb{P}}_N\right)$ 当且仅当 $\hat{\mathbb{P}}_N$ 弱收敛到 $\mathbb{P}$ 并且 $\mathbb{E}_{\hat{P}_N}\left[d\left(Z, \hat{z}_0\right)^p\right]$ 收敛到 $\mathbb{E}_{\mathbb{P}}\left[d\left(Z, \hat{z}_0\right)^p\right]$ 时收敛到 0（Villani 2008，第6.9定理）。此外，$p$-Wasserstein 距离具有吸引力的测度集中性质。具体来说，如果 $\hat{\mathbb{P}}_N$ 表示从 $\mathbb{P}$ 中独立抽取的 $N$ 个样本所得到的经验分布，则 $\hat{\mathbb{P}}_N$ 收敛到 $\mathbb{P}$ 的 $p$-Wasserstein 距离的速率存在严格的渐近和有限样本界限（Fournier 和 Guillin 2015，Weed 和 Bach 2019）。

由于 $p$-Wasserstein 距离是最优运输差异的 $p$ 次方根，命题 2.16 和推论 2.17 自然而然地意味着它也有对偶表示。对于 $p=1$，这个对偶表示特别简单。事实上，可以证明，1-Wasserstein 距离与由所有相对于度量 $d$ 是 Lipschitz 连续且 Lipschitz 模数至多为 1 的测试函数生成的积分概率度量相一致。

推论 2.19 (Kantorovich-Rubinstein 对偶性)。我们有：

$$
\mathrm{W}_1(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{L}^1(\mathbb{P}), \operatorname{lip}(f) \leq 1} \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} f(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) .
$$

证明。推论 2.17 意味着

$$
\mathrm{W}_1(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{L}^1(\mathbb{P})} \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} f^c(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z})
$$

此外，它确保，如果我们将搜索空间限制为可表示为 $f=g^c$ 的函数，其中 $g: \mathcal{Z} \rightarrow(-\infty,+\infty]$，则上确界不变。根据（2.26），我们有 $f(z)=\inf _{\hat{z} \in \mathcal{Z}} g(\hat{z})-d(z, \hat{z})$。对于任何固定的 $\hat{z} \in \mathcal{Z}$，辅助函数 $f_{\hat{z}}(z)=g(\hat{z})-d(z, \hat{z})$ 显然是相对于度量 $d$ 的 1-Lipschitz 函数。由于 1-Lipschitz 函数的下确界仍然是 1-Lipschitz 函数，因此我们得到 $\operatorname{lip}(f) \leq 1$。总之，我们已经证明了，限制注意力集中在 1-Lipschitz 函数上不会减少对偶最优运输问题的上确界。接下来，我们证明 $\operatorname{lip}(f) \leq 1$ 蕴含 $f^c=f$。事实上，对于任何 $\hat{z} \in \mathcal{Z}$，我们有：

$$
f(\hat{z}) \leq \sup _{z \in \mathcal{Z}} f(z)-d(z, \hat{z}) \leq \sup _{z \in \mathcal{Z}} f(\hat{z})+d(z, \hat{z})-d(z, \hat{z})=f(\hat{z}),
$$

由于两个不等式分别成立，因为 $d(\hat{z}, \hat{z})=0$ 和 $\operatorname{lip}(f) \leq 1$。这通过（2.25）意味着对于所有 $\hat{z} \in \mathcal{Z}$，$f(\hat{z})=\sup _{z \in \mathcal{Z}} f(z)-d(z, \hat{z})=f^c(\hat{z})$。因此，当 $\operatorname{lip}(f) \leq 1$ 时，$f^c$ 与 $f$ 一致，因此该命题成立。

$p$-Wasserstein 模糊集的半径为 $r \geq 0$，围绕参考分布 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 定义为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\} .
$$

Pflug、Pichler 和 Wozabal（2012）研究了稳健投资组合选择问题，其中资产收益分布的不确定性由 $p$-Wasserstein 球体表示。他们证明了，当 $r$ 趋近于无穷大时，最优策略是将资本均等分配给所有可用资产。因此，这一结果揭示了在极端模糊下，实践中广泛使用的 $1 / N$ 投资策略（DeMiguel, Garlappi 和 Uppal 2009）是最优的。Pflug 等人（2012）、Pichler（2013）和 Wozabal（2014）进一步证明，对于广泛的凸风险度量，在 $p$-Wasserstein 球体内所有分布的最坏情况投资组合风险等于参考分布下的名义风险，加上一个随着 Wasserstein 半径 $r$ 扩展的正则化项；参见第8.3节。


### 2.3.1. $p$-Wasserstein模糊集

$p$-Wasserstein 距离在DRO中具有特别的重要性。Kantorovich-Rubinstein 对偶性可用于构造1-Wasserstein球体内所有分布的Lipschitz连续损失函数的最坏期望的简单上界。这个上界是由名义分布 $\hat{\mathbb{P}}$ 下的期望损失加上一个正则化项组成，正则化项由损失函数的Lipschitz模数和模糊集的半径 $r$ 加权而成。Shafieezadeh-Abadeh、Mohajerin Esfahani 和 Kuhn (2015) 证明了这个上界在分布稳健的逻辑回归问题中是准确的。然而，实际上这一准确性结果扩展到了许多线性预测模型，且损失函数是凸的（Chen 和 Paschalidis 2018, 2019, Blanchet, Kang 和 Murthy 2019b, Shafieezadeh-Abadeh 等 2019, Wu, Li 和 Mao 2022），甚至是非凸的损失函数（Gao 等 2024, Ho-Nguyen 和 Wright 2023）。更一般地，1-Wasserstein 模糊集在不同领域中得到了广泛应用，如二阶段和多阶段随机规划（Zhao 和 Guan 2018, Hanasusanto 和 Kuhn 2018, Duque 和 Morton 2020, Bertsimas, Shtern 和 Sturt 2023）、机会约束规划（Chen, Kuhn 和 Wiesemann 2024b, Xie 2021, Ho-Nguyen, Kılınç-Karzan, Küçükyavuz 和 Lee 2022, Shen 和 Jiang 2023）、逆优化（Mohajerin Esfahani, Shafieezadeh-Abadeh, Hanasusanto 和 Kuhn 2018）、统计学习（Blanchet, Glynn, Yan 和 Zhou 2019a, Zhu, Xie, Zhang, Gao 和 Xie 2022b）、假设检验（Gao, Xie, Xie 和 Xu 2018）、上下文随机优化（Zhang, Yang 和 Gao 2024a）、运输（Sun, Xie 和 Witten 2023）、控制（Cherukuri 和 Cortés 2019, Yang 2020, Boskos, Cortés 和 Martínez 2020, Li 和 Martínez 2020, Coulson, Lygeros 和 Dörfler 2021, Aolaritei, Lanzetti, Chen 和 Dörfler 2022a, Terpin, Lanzetti, Yardim, Dörfler 和 Ramponi 2022, Terpin, Lanzetti 和 Dörfler 2024）、电力系统分析（Wang, Gao, Wei, Shafie-khah, Bi 和 Catalao 2018, Ordoudis, Nguyen, Kuhn 和 Pinson 2021）等。

$p$-Wasserstein 距离（特别是 $p=2$）也有着广泛的应用。在回顾其各种用途之前，我们强调2-Wasserstein 距离与在2.1.4节中介绍的Gelbrich距离之间的有趣联系（参见定义2.1）。正如Gelbrich（1990，第2.1定理）指出的那样，两个概率分布之间的2-Wasserstein 距离提供了它们均值协方差对之间Gelbrich 距离的上界。

**定理 2.20** (Gelbrich界限)。假设$\mathcal{Z}$ 配备了欧几里得度量 $d(z, \hat{z})=\|z-\hat{z}\|_2$。对于任意具有有限均值向量 $\mu, \hat{\mu} \in \mathbb{R}^d$ 和协方差矩阵 $\Sigma, \hat{\Sigma} \in \mathbb{S}_{+}^d$ 的分布 $\mathbb{P}, \hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$，我们有：

$$
\mathrm{W}_2(\mathbb{P}, \hat{\mathbb{P}}) \geq \mathrm{G}((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma})) .
$$

证明。根据定义，平方的2-Wasserstein 距离满足：

$$
\begin{aligned}
& \mathrm{W}_2^2(\mathbb{P}, \hat{\mathbb{P}})=\inf _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})} \int_{\mathcal{Z} \times \mathcal{Z}}\|z-\hat{z}\|_2^2 \mathrm{~d} \gamma(z, \hat{z}) \\
& = \begin{cases}\text { inf } & \|\mu-\hat{\mu}\|_2^2+\operatorname{Tr}[\Sigma+\hat{\Sigma}-2 C] \\
\text { s.t. } & \gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}}), C \in \mathbb{R}^{d \times d} \\
& \int_{\mathcal{Z} \times \mathcal{Z}}\left[\begin{array}{l}
z-\mu \\
\hat{z}-\hat{\mu}
\end{array}\right]\left[\begin{array}{c}
z-\mu \\
\hat{z}-\hat{\mu}
\end{array}\right]^{\top} \mathrm{d} \gamma(z, \hat{z})=\left[\begin{array}{cc}
\Sigma & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right], \quad\left[\begin{array}{cc}
\Sigma & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0 .\end{cases}
\end{aligned}
$$

注意，新的决策变量 $C$ 是由运输计划 $\gamma$ 唯一确定的，也就是它表示 $Z$ 和 $\hat{Z}$ 在 $\gamma$ 下的交叉协方差矩阵。因此，它的存在并不会扩大可行集。还要注意，最后表达式中的线性矩阵不等式是多余的，因为 $\gamma$ 的二阶矩阵必然是半正定的。因此，它的存在不会减少可行集。最后，注意到二次函数的积分：

$$
\begin{aligned}
\|z-\hat{z}\|_2^2= & \|\mu-\hat{\mu}\|_2^2+\|z-\mu\|_2^2+\|\hat{z}-\hat{\mu}\|_2^2-2(z-\mu)^{\top}(\hat{z}-\hat{\mu}) \\
& +2(\mu-\hat{\mu})^{\top}(z-\mu)-2(\mu-\hat{\mu})^{\top}(\hat{z}-\hat{\mu})
\end{aligned}
$$

相对于 $\gamma$ 的积分是唯一由 $\gamma$ 的一阶和二阶矩确定的，并且它等于 $\|\mu-\hat{\mu}\|_2^2+\operatorname{Tr}[\Sigma+\hat{\Sigma}-2 C]$。通过放宽最后的优化问题，移除所有涉及 $\gamma$ 的约束，从而得到：

$$
\mathrm{W}_2^2(\mathbb{P}, \hat{\mathbb{P}}) \geq\left\{\begin{array}{cl}
\min _{C \in \mathbb{R}^{d \times d}} & \|\mu-\hat{\mu}\|_2^2+\operatorname{Tr}[\Sigma+\hat{\Sigma}-2 C] \\
\text { s.t. } & {\left[\begin{array}{cc}
\Sigma & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0}
\end{array}\right.
$$

根据命题 2.2，得到的半正定规划的最优值即为 $\mathrm{G}^2((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma}))$。通过对两边开方，得出结论。

定理 2.20 的证明揭示了平方的Gelbrich 距离与一个松弛的最优运输问题的最小值一致，这个松弛问题只要求运输计划 $\gamma$ 的边际具有与 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 分别相同的一阶和二阶矩。Gelbrich 不等式在无法精确计算2-Wasserstein 距离时可能会很有用。事实上，在离散分布只有两个原子时，计算离散分布和连续分布之间的2-Wasserstein 距离就已经是 #P-hard（Taşkesen, Shafieezadeh-Abadeh 和 Kuhn 2023a）。当两个分布都是离散时，计算2-Wasserstein 距离甚至可能是 #P-hard（Taşkesen, Shafieezadeh-Abadeh, Kuhn 和 Natarajan 2023b）。如果 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 都是高斯分布，那么Gelbrich 不等式就变为等式。因此，两个高斯分布之间的2-Wasserstein 距离与它们的均值向量和协方差矩阵之间的Gelbrich 距离相等（Givens 和 Shortt 1984，第7条命题）。这个经典结果实际上早于Gelbrich 不等式，如今被认为是Brenier（1991）提出的最优运输问题最优性的直接结果。利用Brenier的最优性条件，可以更一般地证明，如果 $\hat{\mathbb{P}}$ 是 $\mathbb{P}$ 的正半定仿射推前，即存在一个仿射函数 $f(z)=A z+b$，其中 $A \in \mathbb{S}_{+}^d$ 和 $b \in \mathbb{R}^d$ 使得 $\hat{\mathbb{P}}=\mathbb{P} \circ f^{-1}$，那么 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 之间的2-Wasserstein 距离再次等于它们的均值向量和协方差矩阵之间的Gelbrich 距离（Nguyen 等 2021，第2条定理）。2-Wasserstein 模糊集已广泛应用于机器学习（Sinha, Namkoong 和 Duchi 2018, Blanchet 等 2019b, Blanchet, Murthy 和 Si 2022b, Blanchet, Murthy 和 Zhang 2022c）、逆优化（Mohajerin Esfahani 等 2018）、二阶段随机规划（Hanasusanto 和 Kuhn 2018）、估计和滤波（Shafieezadeh-Abadeh 等 2018, Nguyen 等 2023, Kargin 等 2024b）、投资组合优化（Blanchet, Chen 和 Zhou 2022a, Nguyen 等 2021）以及控制理论（Al Taha 等 2023, Hajar 等 2023, Hakobyan 和 Yang 2024, Taşkesen 等 2024, Kargin 等 2024a,c,d）。


### 2.3.2. Lévy-Prokhorov模糊集

Lévy-Prokhorov距离是最广泛使用的概率度量之一，因为它对$\mathcal{P}(\mathcal{Z})$上的弱收敛拓扑进行了度量化。下面我们假设$d(\cdot, \cdot)$是$\mathcal{Z}$上的一个连续度量。对于任何集合$\mathcal{B} \subseteq \mathcal{Z}$和$r \geq 0$，我们使用

$$
\mathcal{B}_r=\left\{z \in \mathcal{Z}: \exists z^{\prime} \in \mathcal{B} \text { with } d\left(z, z^{\prime}\right) \leq r\right\}
$$

来表示集合$\mathcal{B}$的$r$-邻域。由于度量$d$通常是上下文中显而易见的，所以在符号上省略了$\mathcal{B}_r$对度量$d$的依赖。通过这些准备，我们现在可以定义Lévy-Prokhorov距离。

**定义2.21**（Lévy-Prokhorov距离）。对于$\mathcal{Z}$上的任意度量$d(\cdot, \cdot)$，由$d$引出的Lévy-Prokhorov距离$\mathrm{LP}: \mathcal{P}(\mathcal{Z}) \times \mathcal{P}(\mathcal{Z}) \rightarrow[0,1]$定义为：

$$
\operatorname{LP}(\mathbb{P}, \hat{\mathbb{P}})=\inf \left\{r \geq 0: \mathbb{P}(\mathcal{B}) \leq \hat{\mathbb{P}}\left(\mathcal{B}_r\right)+r \text { for all Borel sets } \mathcal{B} \subseteq \mathcal{Z}\right\},
$$

其中$\mathcal{B}_r$在(2.29)中定义。
Lévy-Prokhorov距离的范围是[0,1]，并且当且仅当其参数相等时，它的值为0。此外，可以轻松证明它满足三角不等式。然而，它似乎是非对称的。下面的命题揭示了Lévy-Prokhorov距离与最优运输理论的紧密联系。

**命题2.22**（Strassen 1965）。如果对于$r \geq 0$，传输成本函数$c_r$通过$c_r(z, \hat{z})=\mathbb{1}_{d(z, \hat{z})>r}$定义，那么有：

$$
\operatorname{LP}(\mathbb{P}, \hat{\mathbb{P}})=\inf \left\{r \geq 0: \mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\} .
$$

证明。注意到$c_r$是下半连续的，因为假设度量$d$是连续的。根据命题2.16，$\mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}})$因此具有如下对偶表示：

$$
\begin{array}{cl}
\sup _{f \in \mathcal{L}^1(\mathbb{P}), g \in \mathcal{L}^1(\hat{\mathbb{P}})} & \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} g(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) \\
\text { s.t. } & f(z)-g(\hat{z}) \leq \mathbb{1}_{d(z, \hat{z})>r} \quad \forall z, \hat{z} \in \mathcal{Z}
\end{array}
$$

这里，对于任何固定的$g$，最优地推高$f$，使得对于所有$z \in \mathcal{Z}$，我们有：

$$
f(z)=\inf _{\hat{z} \in \mathcal{Z}} g(\hat{z})+\mathbb{1}_{d(z, \hat{z})>r} \Longrightarrow \inf _{\hat{z} \in \mathcal{Z}} g(\hat{z}) \leq f(z) \leq 1+\inf _{\hat{z} \in \mathcal{Z}} g(\hat{z}) .
$$

同样，对于任何固定的$f$，最优地推低$g$，使得对于所有$\hat{z} \in \mathcal{Z}$，我们有：

$$
g(\hat{z})=\sup _{z \in \mathcal{Z}} f(z)-\mathbb{1}_{d(z, \hat{z})>r} \Longrightarrow \sup _{z \in \mathcal{Z}} f(z)-1 \leq g(\hat{z}) \leq \sup _{z \in \mathcal{Z}} f(z) .
$$
结合(2.31b)中$g(\hat{z})$的上界与(2.31a)中$f(z)$的上界，进一步推导出$g(\hat{z}) \leq \sup _{z \in \mathcal{Z}} f(z) \leq 1+\inf _{z^{\prime} \in \mathcal{Z}} g\left(z^{\prime}\right)$。最优时，(2.31a)和(2.31b)必须同时成立，因此我们得到：

$$
\inf _{z^{\prime} \in \mathcal{Z}} g\left(z^{\prime}\right) \leq f(z) \leq 1+\inf _{z^{\prime} \in \mathcal{Z}} g\left(z^{\prime}\right) \quad \text { and } \quad \inf _{z^{\prime} \in \mathcal{Z}} g\left(z^{\prime}\right) \leq g(\hat{z}) \leq 1+\inf _{z^{\prime} \in \mathcal{Z}} g\left(z^{\prime}\right)
$$

对于所有$z, \hat{z} \in \mathcal{Z}$。注意到，由于$\mathbb{P}$和$\hat{\mathbb{P}}$都是概率分布，对偶最优运输问题(2.30)的目标函数在替换$f(z) \leftarrow f(z)-\inf _{z^{\prime} \in \mathcal{Z}} g\left(z^{\prime}\right)$和$g(\hat{z}) \leftarrow g(\hat{z})-\inf _{z^{\prime} \in \mathcal{Z}} g\left(z^{\prime}\right)$时保持不变。因此，以下我们可以在不失一般性的情况下假设，对于所有$z \in \mathcal{Z}$，有$0 \leq f(z) \leq 1$，对于所有$\hat{z} \in \mathcal{Z}$，有$0 \leq g(\hat{z}) \leq 1$。

由于$f$和$g$现在被归一化到$[0,1]$，它们具有以下积分表示：

$$
f(z)=\int_0^1 \mathbb{1}_{f(z) \geq \tau} \mathrm{d} \tau \quad \forall z \in \mathcal{Z} \quad \text { and } \quad g(\hat{z})=\int_0^1 \mathbb{1}_{g(\hat{z}) \geq \tau} \mathrm{d} \tau \quad \forall \hat{z} \in \mathcal{Z} .
$$

接下来，可以证明，$f$和$g$当且仅当满足以下约束时，在(2.30)中是可行的：

$$
\mathbb{1}_{f(z) \geq \tau}-\mathbb{1}_{g(\hat{z}) \geq \tau} \leq \mathbb{1}_{d(z, \hat{z})>r} \quad \forall z, \hat{z} \in \mathcal{Z}, \forall \tau \in[0,1]
$$

首先注意，(2.32)在其左侧计算为1且右侧计算为0时是平凡满足的。这仅在某些$\tau \in[0,1]$和$z, \hat{z} \in \mathcal{Z}$满足$f(z) \geq \tau$且$g(\hat{z}) < \tau$，且$d(z, \hat{z}) \leq r$时才会发生。然而，由于这意味着对于某些$z, \hat{z}$，$f(z)-g(\hat{z}) > 0$，这与(2.30)中的约束相矛盾。因此，(2.30)中的约束意味着(2.32)。反之，由于$f$和$g$的积分表示，(2.32)也立即成立。

最后，注意到$\mathbb{1}_{f(z) \geq \tau}$和$\mathbb{1}_{g(\hat{z}) \geq \tau}$分别是Borel集$\mathcal{B}=\{z \in \mathcal{Z}: f(z) \geq \tau\}$和$\mathcal{C}=\{\hat{z} \in \mathcal{Z}: g(\hat{z}) \geq \tau\}$的特征函数。同样，(2.32)当且仅当$\mathcal{C} \supseteq \mathcal{B}_r$时成立。回顾它们的积分表示，我们可以得出结论，只有当$f$和$g$表示为（无限多个）特征函数的凸组合时，$f$和$g$才是(2.30)的可行解，这些特征函数形如$\mathbb{1}_{z \in \mathcal{B}}$和$\mathbb{1}_{\hat{z} \in \mathcal{C}}$，其中$\mathcal{B}$和$\mathcal{C}$是Borel集，且满足$\mathcal{C} \supseteq \mathcal{B}_r$。由于(2.30)的目标函数是线性的，因此如果我们将可行集限制为这些特征函数，其上确界不会改变。因此，(2.30)简化为：

$$
\mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}})=\sup \left\{\mathbb{P}(\mathcal{B})-\hat{\mathbb{P}}(\mathcal{C}): \mathcal{B}, \mathcal{C} \subseteq \mathcal{Z} \text { are Borel sets with } \mathcal{C} \supseteq \mathcal{B}_r\right\}
$$

显然，始终最优地设置$\mathcal{C}=\mathcal{B}_r$，因此该结论得以证明。

虽然命题2.22来源于(Strassen 1965, Theorem 11)，但这里的证明与(Villani 2003, Theorem 1.27)的证明类似。作为副产品，命题2.22揭示了Lévy-Prokhorov距离是对称的，这在其定义中并不显而易见。因此，它确实是一个度量。

Lévy-Prokhorov模糊集的半径$r \geq 0$围绕着$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$定义为：

$$
\mathcal{P}=\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{LP}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\}
$$

对于我们的目的，命题2.22的最重要含义是$\mathcal{P}$可以看作是一个最优运输模糊集的特例，即对于任何半径$r \geq 0$，我们有：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}
$$

Lévy-Prokhorov模糊集最早是在机会约束规划的背景下引入的（Erdoğan 和 Iyengar 2006）。它们也自然出现在数据驱动决策和训练鲁棒机器学习模型中（Pydi 和 Jog 2021，Bennouna 和 Van Parys 2023，Bennouna，Lucas 和 Van Parys 2023）。我们在本节的结尾给出一个有用的推论，它直接来自命题2.22证明的最后部分。

推论2.23。如果对应于$r \geq 0$的传输成本函数$c_r$通过$c_r(z, \hat{z})=\mathbb{1}_{d(z, \hat{z})>r}$定义，那么我们有：

$$
\mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}})=\sup \left\{\mathbb{P}(\mathcal{B})-\hat{\mathbb{P}}\left(\mathcal{B}_r\right): \mathcal{B} \subseteq \mathcal{Z} \text { is a Borel set }\right\},
$$

其中$r$-邻域$\mathcal{B}_r$在(2.29)中定义。


### 2.2.3. 总变差模糊集

两个分布 $\mathbb{P}, \hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 的总变差距离是 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 在任何事件上所分配的概率之间的最大绝对差异。

**定义 2.10（总变差距离）** 总变差距离是定义为

$$
\operatorname{TV}(\mathbb{P}, \hat{\mathbb{P}}) = \sup \{|\mathbb{P}(\mathcal{B}) - \hat{\mathbb{P}}(\mathcal{B})| : \mathcal{B} \subseteq \mathcal{Z} \text{ 是一个 Borels 集合} \}
$$

总变差距离显然是对称的，满足不可区分的恒等式和三角不等式。因此，它构成了 $\mathcal{P}(\mathcal{Z})$ 上的一个度量。此外，总变差距离是一个 $\phi$-散度的实例。

** 命题 2.11.** 总变差距离与由熵函数 $\phi(s) = \frac{1}{2}|s - 1|$ 对所有 $s \geq 0$ 引起的 $\phi$-散度一致。

**证明：** 诱发 Kullback-Leibler 散度的熵函数的共轭熵函数满足 $\phi^*(t) = \max\{t, -\frac{1}{2}\}$，当 $t \leq \frac{1}{2}$ 时，$\phi^*(t) = +\infty$ 当 $t > \frac{1}{2}$ 时。根据命题 2.6，与给定熵函数对应的 $\phi$-散度因此具有如下的对偶表示：

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) = \sup _{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z) - \int_{\mathcal{Z}} \max\{f(z), -\frac{1}{2}\} \mathrm{d} \hat{\mathbb{P}}(z),
$$

其中 $\mathcal{F}$ 表示所有有界的 Borel 函数 $f: \mathcal{Z} \rightarrow \left(-\infty, \frac{1}{2}\right]$ 的族。注意，$\mathcal{F}$ 对常数平移不变。也就是说，如果 $f(z)$ 是一个有界的 Borel 函数，那么 $f(z) + c$ 对于任何常数 $c \in \mathbb{R}$ 也是有界的 Borel 函数。因此，我们可以在上述最大化问题中同时对 $f \in \mathcal{F}$ 和 $c \in \mathbb{R}$ 进行优化，从而得到：

$$
\operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \sup _{f \in \mathcal{F}} \sup _{c \in \mathbb{R}} \int_{\mathcal{Z}} (f(z) + c) \mathrm{d\mathbb{P}}(z) - \int_{\mathcal{Z}} \left(\mathrm{e}^{f(z) + c} - 1\right) \mathrm{d} \hat{\mathbb{P}}(z).
$$

对于任何固定的 $f \in \mathcal{F}$，关于 $c$ 的内层最大化问题可以通过以下方式唯一求解：

$$
c^{\star} = -\log \left(\int_{\mathcal{Z}} \mathrm{e}^{f(z)} \mathrm{d} \hat{\mathbb{P}}(z)\right).
$$

将此表达式代回目标函数中，得到（2.12）。命题 2.9 建立了 Kullback-Leibler 散度与熵风险度量之间的联系。这个联系将在第 4.3 节中变得有用。

Kullback-Leibler 模糊集的半径为 $r \geq 0$ 的模糊集定义为：

$$
\mathcal{P} = \{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\}.
$$

由于 $\phi^{\infty}(1) = +\infty$，因此所有分布 $\mathbb{P} \in \mathcal{P}$ 都是相对于 $\hat{\mathbb{P}}$ 的绝对连续的。因此，$\mathcal{P}$ 与限制 Kullback-Leibler 模糊集一致。El Ghaoui 等人（2003）推导出了当 $\hat{\mathbb{P}}$ 是高斯分布时，线性损失函数的最坏情况价值风险的封闭形式表达式。Hu 和 Hong（2013）使用类似的技术表明，任何相对于 Kullback-Leibler 模糊集的分布鲁棒性个体机会约束等价于参考分布 $\hat{\mathbb{P}}$ 下的经典机会约束，但具有重新调整的置信水平。Calafiore（2007）研究了当 $\hat{\mathbb{P}}$ 是离散分布时，最坏情况下的均值风险投资组合选择问题。Kullback-Leibler 模糊集还在最小二乘估计（Levy 和 Nikoukhah 2004）、假设检验（Levy 2008, Gül 和 Zoubir 2017）、滤波（Levy 和 Nikoukhah 2012, Zorzi 2016, 2017a,b）、风险度量理论（Ahmadi-Javid 2012, Postek 等人 2016）和极值分析（Blanchet, He 和 Murthy 2020）等领域找到了应用。

半径为 $r \geq 0$ 的总变差模糊集围绕 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 定义为：

$$
\mathcal{P}=\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{TV}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\}
$$

现有文献大多集中在限制性总变差模糊集上，该模糊集包含所有满足 $\mathbb{P} \ll \hat{\mathbb{P}}$ 的分布 $\mathbb{P} \in \mathcal{P}$。Jiang 和 Guan (2018, Theorem 1) 以及 Shapiro (2017, Example 3.7) 证明，相对于限制性总变差模糊集的最坏期望损失与条件价值-at-risk 和相对于 $\hat{\mathbb{P}}$ 的损失的本质上确界的组合一致，详见第6.10节。Rahimian、Bayraksan 和 Homem-deMello (2019a, $b$, 2022) 研究了在 $\mathcal{Z}$ 是有限集的情况下，未限制总变差模糊集上的 DRO 问题的最坏分布。总变差模糊集与鲁棒统计中的 Huber 污染模型相关（Huber 1981），该模型假设统计数据集中的一部分样本（比例为 $r \in(0,1)$）来自于一个任意的污染分布。因此，目标分布和污染数据生成分布之间的总变差距离至多为 $r$。因此，围绕某个估计分布的半径 $r$ 的总变差模糊集作为目标分布的搜索空间是自然的选择（Nishimura 和 Ozaki 2004, 2006；Bose 和 Daripa 2009；Duchi, Hashimoto 和 Namkoong 2023；Tsanga 和 Shehadeha 2024）。


### 2.3.4. $\infty$-Wasserstein模糊集

第2.3.1节专门讨论了对应于有限指数$p \in[1, \infty)$的$p$-Wasserstein距离。$\infty$-Wasserstein距离需要特别的处理。

**定义2.25**（$\infty$-Wasserstein距离）。对于$\mathcal{Z}$上的一个连续度量$d(\cdot, \cdot)$，对应的$\infty$-Wasserstein距离$\mathrm{W}_{\infty}$ : $\mathcal{P}(\mathcal{Z}) \times \mathcal{P}(\mathcal{Z}) \rightarrow[0, \infty]$定义为：

$$
\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})=\inf _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})} \operatorname{ess} \sup _\gamma[d(Z, \hat{Z})],
$$

其中，$d(Z, \hat{Z})$在$\gamma$下的本质上确界由以下给出：

$$
\operatorname{ess} \sup _\gamma[d(Z, \hat{Z})]=\inf _{\tau \in \mathbb{R}}\{\tau: \gamma(d(Z, \hat{Z})>\tau)=0\}
$$

定义2.25是合理的，因为$\infty$-Wasserstein距离可以通过$p$-Wasserstein距离在$p$趋向无穷大时的极限来获得。

**命题2.26**（Givens和Shortt 1984）。对于任何$\mathbb{P}, \hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$，有：

$$
\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})=\lim _{p \rightarrow \infty} \mathrm{~W}_p(\mathbb{P}, \hat{\mathbb{P}})=\sup _{p \geq 1} \mathrm{~W}_p(\mathbb{P}, \hat{\mathbb{P}}) .
$$

证明。如果$p \geq q \geq 1$，则$f(t)=t^{q / p}$在$\mathbb{R}_{+}$上是凹的。这意味着：

$$
\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})=\inf _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})}\left(\mathbb{E}_\gamma\left[d(Z, \hat{Z})^p\right]^{\frac{q}{p}}\right)^{\frac{1}{q}} \geq \inf _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})}\left(\mathbb{E}_\gamma\left[d(Z, \hat{Z})^q\right]\right)^{\frac{1}{q}}=\mathrm{W}_q(\mathbb{P}, \hat{\mathbb{P}})
$$

根据Jensen不等式。因此，$\mathbf{W}_p(\mathbb{P}, \hat{\mathbb{P}})$在指数$p$上是单调递增的，只要$p \in[1, \infty)$。此外，对于任何交通计划$\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})$和指数$p \in[1, \infty)$，本质上确界的定义直接意味着：

$$
\left(\mathbb{E}_\gamma\left[d(Z, \hat{Z})^p\right]\right)^{\frac{1}{p}} \leq \operatorname{ess}^{-1} \sup _\gamma\left[d(Z, \hat{Z})^p\right]^{\frac{1}{p}}=\operatorname{ess} \sup _\gamma[d(Z, \hat{Z})] .
$$

对上述不等式两边在所有$\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})$上进行最小化，进一步推导出$\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}}) \leq \mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})$对于所有$p \in[1, \infty)$成立。因此，我们可以得出结论：

$$
\lim _{p \rightarrow \infty} \mathrm{~W}_p(\mathbb{P}, \hat{\mathbb{P}})=\sup _{p \geq 1} \mathrm{~W}_p(\mathbb{P}, \hat{\mathbb{P}}) \leq \mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}}) .
$$

剩下的任务是证明这个不等式实际上成立为等式。为此，固定一些容忍度$\varepsilon>0$。对于任意$p \in \mathbb{N}$，令$\gamma_p \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})$是一个配对，使得$\mathbb{E}_{\gamma_p}\left[d(Z, \hat{Z})^p\right]^{1 / p}=\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})$。注意到$\gamma_p$是存在的，因为正如我们将在推论3.16和命题3.3中看到的那样，$\Gamma(\mathbb{P}, \hat{\mathbb{P}})$是弱紧的，并且$\mathbb{E}_\gamma\left[d(Z, \hat{Z})^p\right]$在$\gamma$中的弱下半连续性。接下来，令$\left\{\gamma_{p(j)}\right\}_{j \in \mathbb{N}}$是一个子序列，它弱收敛到某个配对$\gamma_{\infty} \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})$，它再次存在，因为$\Gamma(\mathbb{P}, \hat{\mathbb{P}})$是弱紧的。我们通过分情况讨论。

**情况1**. 如果$\operatorname{ess} \sup _{\gamma_{\infty}}[d(Z, \hat{Z})]$是有限的，定义开集：

$$
\mathcal{B}=\left\{(z, \hat{z}) \in \mathcal{Z} \times \mathcal{Z}: d(z, \hat{z})>\operatorname{ess} \sup _{\gamma_{\infty}}[d(Z, \hat{Z})]-\varepsilon\right\},
$$

并注意到根据本质上确界的定义，$\gamma_{\infty}(\mathcal{B})>0$。我们得到：

$$
\begin{aligned}
\mathrm{W}_{p(j)}(\mathbb{P}, \hat{\mathbb{P}}) & \geq\left(\int_{\mathcal{B}} d(z, \hat{z})^{p(j)} \mathrm{d} \gamma_{p(j)}(z, \hat{z})\right)^{\frac{1}{p(j)}} \\
& \geq \gamma_{p(j)}(\mathcal{B})^{\frac{1}{p(j)}}\left(\operatorname{ess} \sup _{\gamma_{\infty}}[d(Z, \hat{Z})]-\varepsilon\right) \\
& \geq \gamma_{p(j)}(\mathcal{B})^{\frac{1}{p(j)}}\left(\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})-\varepsilon\right) .
\end{aligned}
$$

由于$\mathcal{B}$是开放的，并且$\gamma_{p(j)}$弱收敛到$\gamma_{\infty}$，随着$j$增大，Portmanteau定理（Billingsley 2013, Theorem 2.1 (iiv)）意味着$\lim \inf _{j \rightarrow \infty} \gamma_{p(j)}(\mathcal{B}) \geq \gamma_{\infty}(\mathcal{B})>0$。因此，$\gamma_{p(j)}(\mathcal{B})^{1 / p(j)}$随着$j$的增大收敛到1，我们得到：

$$
\lim _{p \rightarrow \infty} \mathrm{~W}_p(\mathbb{P}, \hat{\mathbb{P}}) \geq \mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})-\varepsilon
$$

由于这个不等式对任意容忍度$\varepsilon>0$都成立，因此上述推理最终意味着$\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})$确实收敛到$\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})$，当$p$足够大时。

**情况2**. 如果$\operatorname{ess} \sup _{\gamma_{\infty}}[d(Z, \hat{Z})]=\infty$，则我们将定义中开集$\mathcal{B}$的$\operatorname{ess}_{\sup _{\gamma_{\infty}}}[d(Z, \hat{Z})]$替换为一个任意大的常数。按照情况1的推理，最终揭示$\lim _{p \rightarrow \infty} \mathrm{~W}_p(\mathbb{P}, \hat{\mathbb{P}})=\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})=\infty$。

为了理解命题2.26的直觉，考虑在$\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})$定义中的最优运输问题。如果$p>1$，则运输一单位概率质量从$\hat{z}$到$z$的费用$c(z, \hat{z})=d(z, \hat{z})^p$随着距离$d(z, \hat{z})$的增大呈超线性增长。因此，在最优运输计划中，$\hat{\mathbb{P}}$的部分分布在更远的地方被运输，对$\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})$的贡献更大。此外，随着$p$趋向无穷大，最终只有在最远处运输的分布部分才对$\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})$有影响。更确切地说，只有最大的运输距离才重要，而运输的概率质量数量则不重要。

尽管有命题2.26的存在，$p<\infty$和$p=\infty$阶Wasserstein距离定义中的最优运输问题在根本上是不同的。事实上，如果$p<\infty$，则最优运输问题的目标函数$\mathbb{E}_\gamma\left[d(Z, \hat{Z})^p\right]$是关于运输计划$\gamma$的线性的。而当$p=\infty$时，目标函数$\operatorname{ess} \sup _\gamma[d(Z, \hat{Z})]$在$\gamma$中的函数形式甚至不是凸的，而是准凸的（Jylhä 2015, Lemma 2.2）；参见（Champion, De Pascale和Juutinen 2008）。因此，$\infty$-Wasserstein距离需要更为细致的处理。

接下来的命题将$\infty$-Wasserstein距离与标准的最优运输问题联系起来，因此具有计算上的相关性。

命题2.27。如果对应于$r \geq 0$的交通成本函数$c_r$通过$c_r(z, \hat{z})=\mathbb{1}_{d(z, \hat{z})>r}$定义，那么我们有：

$$
\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})=\inf \left\{r \geq 0: \mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}}) \leq 0\right\}
$$

证明。回顾$\mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}})=\inf \left\{\mathbb{E}_\gamma\left[c_r(Z, \hat{Z})\right]: \gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})\right\}$。注意到基础的最优运输问题是可解的，因为$\Gamma(\mathbb{P}, \hat{\mathbb{P}})$是弱紧的，并且$\mathbb{E}_\gamma\left[d(Z, \hat{Z})^p\right]$在$\gamma$中的弱下半连续性得益于推论3.16和命题3.3。由于此，得出：

$$
\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})=\inf \left\{r \geq 0: \mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}}) \leq 0\right\}
$$

命题2.27结合推论2.23立即得出$\infty$-Wasserstein距离的以下等效表述。

推论2.28（Givens和Shortt 1984）。$\infty$-Wasserstein距离满足：

$$
\mathbf{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})=\inf \left\{r \geq 0: \mathbb{P}(\mathcal{B}) \leq \hat{\mathbb{P}}\left(\mathcal{B}_r\right) \text { for all Borel sets } \mathcal{B} \subseteq \mathcal{Z}\right\},
$$

其中$r$-邻域$\mathcal{B}_r$在(2.29)中定义。

$\infty$-Wasserstein模糊集的半径$r \geq 0$围绕着$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$定义为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}
$$

命题2.27表明，$\mathcal{P}$与交通成本函数$c_r(z, \hat{z})=\mathbb{1}_{d(z, \hat{z})>r}$的最优运输模糊集重合，即我们有：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}}) \leq 0\right\}
$$

带有$\infty$-Wasserstein模糊集的DRO与对抗性机器学习（Gao, Chen和Kleywegt 2017, García Trillos和García Trillos 2022, García Trillos和Murray 2022, García Trillos和Jacobs 2023, Bungert, García Trillos和Murray 2023, Bungert, Laux和Stinson 2024, Gao等 2024, Pydi和Jog 2024, Frank和Niles-Weed 2024a,b）以及核密度估计（Xu, Caramanis和Mannor 2012a）有着密切的联系。此外，$\infty$-Wasserstein模糊集还被用于二阶段和多阶段随机规划（Xie 2020, Bertsimas, Shtern和Sturt 2022, Bertsimas等 2023）、投资组合优化（Nguyen, Zhang, Wang, Blanchet, Delage和Ye 2024）和鲁棒学习（Nguyen, Zhang, Blanchet, Delage和Ye 2020, Wang, Nguyen和Hanasusanto 2024c）等多个领域。


## 2.4. 其他模糊集

存在一些无法归类为矩 moment、$\phi$-散度或最优运输模糊集的模糊集。以下是对这些模糊集的简要概述，未提供详细的数学细节。


### 2.4.1. 边缘模糊集

边缘模糊集指定多个子向量的边缘分布，而不详细描述它们的联合分布。边缘模糊集的最简单例子是Fréchet模糊集，它指定了所有个体组成部分$Z$的边缘分布，但未提供它们的联合分布信息。因此，Fréchet模糊集通过$d$个边缘累计分布函数$F_i: \mathbb{R} \rightarrow[0,1], i \in[d]$来参数化，并可以表示为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right): \mathbb{P}\left(Z_i \leq z_i\right)=F_i\left(z_i\right) \forall z_i \in \mathbb{R}, \forall i \in[d]\right\} .
$$

这里，$F_i$是任意的累计分布函数，即一个右连续、单调递增的函数，满足$\lim _{z_i \rightarrow-\infty} F_i\left(z_i\right)=0$和$\lim _{z_i \rightarrow+\infty} F_i\left(z_i\right)=1$。Fréchet模糊集对于概率逻辑具有重要意义。假设每个$Z_i$表示一个二值变量，如果某个事件发生，则评估为1，否则为0，并且假设每个事件的概率是已知的，而所有事件的联合分布未知。在这种情境下，Boole（1854）曾感兴趣的是计算由$Z_i, i \in[d]$的布尔函数编码的复合事件的概率界限。近一个世纪后，Fréchet（1935）推导出了此类复合事件的概率的显式不等式，现在称为Fréchet不等式。注意，这些Fréchet不等式可以通过在具有伯努利边缘的Fréchet模糊集中的所有分布上最小化或最大化复合事件的概率来获得。近年来，广义Fréchet不等式受到了越来越多的关注，这些不等式界定了在Fréchet模糊集内所有分布下，$Z$的广义（不一定是布尔）函数的风险。例如，在金融和风险管理中，已经出现了大量针对随机变量和其和的Fréchet不等式（Rüschendorf 1983, 1991, Embrechts和Puccetti 2006, Wang和Wang 2011, Wang, Peng和Yang 2013, Puccetti和Rüschendorf 2013, Van Parys, Goulart和Embrechts 2016a, Blanchet, Lam, Liu和Wang 2024a）。此外，Natarajan, Song和Teo（2009b）推导出了Fréchet模糊集上的分段仿射函数的最坏期望的精确界限。我们还要强调，Fréchet模糊集还很重要，因为它们与多边缘最优运输问题的可行集一致，而这些问题有时可以在多项式时间内求解（Pass 2015, Altschuler和Boix-Adsera 2023, Natarajan, Padmanabhan和Ramachandra 2023）。

一般边缘模糊集指定了多个（可能重叠）子集$\left\{Z_i: i \in[d]\right\}$的边缘分布。然而，即使每个$Z_i$是一个伯努利随机变量，并且每个子集仅包含两个元素，检查此类模糊集是否非空也是NP完全的（Honeyman, Ladner和Yannakakis 1980, Georgakopoulos, Kavvadias和Papadimitriou 1988）。因此，计算边缘模糊集上的最坏期望是不可行的，除非已知边缘的随机变量子集是互不相交的（Doan和Natarajan 2012），或者相应的重叠图具有运行交集性质（Doan, Li和Natarajan 2015）。边缘模糊集具有吸引力，因为在有限的统计数据下，估计低维边缘比其全局依赖结构容易得多。然而，即使是单变量边缘，也无法精确估计。因此，许多研究者研究了只提供有限信息（例如边缘矩或边缘离散度量界限）的边缘模糊集（Bertsimas等 2004, Bertsimas, Natarajan和Teo 2006a,b, Chen, Sim, Sun和Teo 2010, Mishra, Natarajan, Tao和Teo 2012, Natarajan, Sim和Uichanco 2018）。

相关文献流关注的是随机变量$Z_i, i \in[d]$是独立的且服从模糊边缘分布的模糊集。例如，Hoeffding模糊集包含所有在一个盒子中的联合分布，具有独立的（且完全未知的）边缘，而Bernstein模糊集包含所有在Hoeffding模糊集中的分布，且服从边缘矩的界限（Nemirovski和Shapiro 2007, Hanasusanto等 2015a）。对每个边缘同时施加均值和均值绝对偏差界限的Bernstein模糊集被用来为基于模糊分布的机会约束程序（Postek, Ben-Tal, den Hertog和Melenberg 2018）、二阶段整数规划（Postek等 2018, Postek, Romeijnders, den Hertog和van der Vlerk 2019）以及排队系统（Wang, Prasad, Hanasusanto和Hasenbein 2024d）推导出安全的可处理逼近。

基于边缘模糊集的DRO与风险管理中的子模性理论和共单调性理论密切相关（Tchen 1980, Rüschendorf 2013, Bach 2013, 2019, Natarajan等 2023, Long, Qi和Zhang 2024）。它在从离散选择建模（Natarajan等 2009b, Mishra, Natarajan, Padmanabhan, Teo和Li 2014, Chen, Ma, Natarajan, Simchi-Levi和Yan 2022, Ruan, Li, Murthy和Natarajan 2022）、排队理论（van Eekelen, den Hertog和van Leeuwaarden 2022）、交通运输（Wang, Chen和Liu 2020, Shehadeh 2023）、机会约束规划（Xie, Ahmed和Jiang 2022）、调度（Mak等 2015）、库存管理（Liu, Chen, Wang和Wang 2024a）、复杂网络分析（Chen, Padmanabhan, Lim和Natarajan 2020, Van Leeuwaarden和Stegehuis 2021, Brugman等 2022）到机制设计（Carroll 2017, Gravin和Lu 2018, Chen等 2024a, Wang, Liu和Zhang 2024b, Wang 2024）等众多应用中具有广泛的应用。有关更多详细信息，请参阅Natarajan（2021）的全面专著。

### 2.4.2. 混合模糊集与结构模糊集

设$\Theta \subseteq \mathbb{R}^m$为一个Borel集合，$\mathbb{P}_\theta \in \mathcal{P}(\mathcal{Z})$为一个由$\theta \in \Theta$唯一确定的参数化分布。假设对于每个固定的Borel集合$\mathcal{B} \subseteq \mathcal{Z}$，$\mathbb{P}_\theta(Z \in \mathcal{B})$是$\theta$的Borel可测函数。然后，参数化的分布族$\left\{\mathbb{P}_\theta: \theta \in \Theta\right\}$可以作为一个混合分布族，进而引入混合模糊集：

$$
\mathcal{P}=\left\{\int_{\Theta} \mathbb{P}_\theta \mathrm{d} \mathbb{Q}(\theta): \mathbb{Q} \in \mathcal{P}(\Theta)\right\} .
$$

因此，$\mathcal{P}$包含所有可以表示为$\mathbb{P}_\theta, \theta \in \Theta$的混合分布。换句话说，对于每个$\mathbb{P} \in \mathcal{P}$，存在一个混合分布$\mathbb{Q} \in \mathcal{P}(\Theta)$，使得对于所有Borel集合$\mathcal{B} \subseteq \mathcal{Z}$，$\mathbb{P}(Z \in \mathcal{B})=\int_{\Theta} \mathbb{P}_\theta(Z \in \mathcal{B}) \mathrm{d} \mathbb{Q}(\theta)$。这一构造确保了$\mathcal{P} \subseteq \mathcal{P}(\mathcal{Z})$是凸的。例如，如果$\mathbb{P}_{\boldsymbol{\theta}}$是一个高斯分布，其均值和协方差矩阵由$\theta$编码，则$\mathcal{P}$包含（可能是连续的）高斯分布的混合。Lasserre和Weisser（2021）研究了与紧参数集$\Theta$对应的混合模糊集，提出了基于半正定规划的逐步逼近方法，用于解决分布鲁棒机会约束的问题。

注意，$\mathcal{P}$可以视为参数化分布族$\left\{\mathbb{P}_\theta: \theta \in \Theta\right\}$的凸包。Minkowski的经典凸分析结果表明，任何Euclidean向量空间中的紧凸子集都与其极点的凸包一致。Choquet理论（Phelps 1965）寻求类似的极点表示，用于拓扑向量空间中的紧凸子集。例如，如果$\left\{\mathbb{P}_\theta: \theta \in \Theta\right\}$是弱紧凸模糊集$\mathcal{P}$的所有极分布集合，则（2.36）构成了$\mathcal{P}$的Choquet表示。

具有某些结构性质的分布族有时承认形式为（2.36）的Choquet表示。例如，设$\mathcal{P}$为所有关于原点对称的分布$\mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right)$的族。这意味着，对于每个Borel集合$\mathcal{B} \subseteq \mathbb{R}^d$，$\mathbb{P}(Z \in \mathcal{B})=\mathbb{P}(-Z \in \mathcal{B})$。然后，可以证明，$\mathcal{P}$的所有极分布都可以表示为$\mathbb{P}_\theta=\frac{1}{2} \delta_{+\theta}+\frac{1}{2} \delta_{-\theta}$，其中$\theta \in \mathbb{R}^d$。因此，$\mathcal{P}$承认形式为（2.36）的Choquet表示。作为另一个例子，设$\mathcal{P}$为所有关于原点$\alpha$-单峰的分布$\mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right)$的族，某些$\alpha>0$。这意味着，对于每个Borel集合$\mathcal{B} \subseteq \mathcal{R}^d$，$t^\alpha \mathbb{P}(Z \in \mathcal{B} / t)$在$t>0$时是单调递增的。然后，可以证明，$\mathcal{P}$的每个极分布都是一种分布$\mathbb{P}_\theta$，其支持在从0到$\theta \in \mathbb{R}^d$的线段上，具有$\mathbb{P}_\theta\left(\|Z\|_2 \leq t\|\theta\|_2\right)=t^\alpha$，对于所有$t \in[0,1]$。因此，$\mathcal{P}$再次承认形式为（2.36）的Choquet表示。我们还注意到，$\mathbb{R}^d$上的$d$-单峰分布也被称为星型单峰分布。人们可以轻松验证，如果一个具有连续概率密度函数的分布是星型单峰的，当且仅当其密度函数沿每条从原点发出的射线单调递减。此外，人们可以证明，所有$\alpha$-单峰分布的族以精确的方式收敛到所有可能分布的族，随着$\alpha$趋于无穷大。有关结构分布族及其Choquet表示的更多信息，请参见（Dharmadhikari和Joag-Dev 1988）。

第2.1节中的矩模糊集已知包含仅具有少数原子的离散分布；参见第7节。然而，真实的物理、技术或经济系统中遇到的不确定性不太可能遵循这些离散分布。相反，它们通常被期望是单峰的。因此，消除矩模糊集中的病态离散分布的有效方法是将其与某个$\alpha>0$的单峰分布的结构模糊集交集。Popescu（2005）结合了Choquet理论和求和平方多项式优化的思想，通过逐步逼近的方法，为求解基于此交集模糊集的最坏期望问题提供了一个层次化的可处理上界，每个上界都是通过求解一个可解的半正定程序得到的。Van Parys, Goulart和Kuhn（2016b）以及Van Parys, Goulart和Morari（2019）扩展了这一方法，并为基于所有$\alpha$-单峰分布的切比雪夫模糊集上的多面体的最坏情况概率和分段线性凸损失函数的最坏情况条件价值-at-risk提供了准确的半正定规划重构；参见（Hanasusanto, Roitch, Kuhn和Wiesemann 2015b）。Li, Jiang和Mathieu（2019a）证明，这些半正定规划重构有时可以简化为高效的二阶锥规划。Li, Jiang和Mathieu（2016）在电力系统应用的背景下证明，补充结构信息通常会导致较少保守的DRO模型。Lam, Liu和Zhang（2021）考虑了分布形状的另一个基本概念——正交单峰性，并建立了相应的Choquet表示，用于解决多变量极端事件估计。最近，Lam, Liu和Singham（2024）将Choquet理论与重要性采样和似然比技术相结合，用于建模分布形状。


### 2.4.3. 非标准$\phi$-散度和最优传输模糊集

已提出大量非标准$\phi$-散度和最优传输差异度量，用于衡量概率分布之间的不相似性。这些方法在设计具有互补计算和统计特性的模糊集时提供了极大的灵活性。非标准距离度量特别包括平滑$\phi$-散度（Zeitouni和Gutman 1991，Yang和Chen 2018，Liu，Van Parys和Lam 2023），以及$\phi$-散度和最优传输差异的组合（Reid和Williamson 2011，Dupuis和Mao 2022，Van Parys 2024）。此外，它们还包括一致性Wasserstein距离（Li和Mao 2022）和Sinkhorn散度（Wang，Gao和Xie 2021），以及基于因果最优传输的散度（Analui和Pflug 2014，Pflug和Pichler 2014，Yang，Zhang，Chen，Gao和Hu 2022，Arora和Gao 2022，Jiang和Obloj 2024），抗离群值最优传输（Nietert，Goldfeld和Shafiee 2024a,b），混合特征最优传输（Selvi，Belbasi，Haugh和Wiesemann 2022，Belbasi，Selvi和Wiesemann 2023），基于聚类的最优传输（Wang，Becker，Van Parys和Stellato 2022），部分最优传输（Esteban-Pérez和Morales 2022），切片最优传输（Olea，Rush，Velez和Wiesel 2022），多边际最优传输（Lau和Liu 2022，García Trillos，Jacobs和Kim 2023，Rychener，Esteban-Pérez，Morales和Kuhn 2024），以及约束条件矩最优传输（Li等 2022，Blanchet，Kuhn，Li和Taşkesen 2023，Sauldubois和Touzi 2024）。


### 2.4.4. 基于积分概率度量的模糊集

设$\mathcal{F}$为一族Borel可测的测试函数$f: \mathcal{Z} \rightarrow \mathbb{R}$，使得当且仅当$f \in \mathcal{F}$时，$-f \in \mathcal{F}$。由$\mathcal{F}$生成的积分概率度量定义为

$$
\mathrm{D}_{\mathcal{F}}(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \mathrm{dP}(z)-\int_{\mathcal{Z}} f(\hat{z}) \mathrm{d} \mathbb{P}(\hat{z})
$$

对于所有分布$\mathbb{P}, \hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$，在这些分布下，所有测试函数$f \in \mathcal{F}$均可积。上述最大化问题探讨了测试函数如何区分$\mathbb{P}$与$\hat{\mathbb{P}}$。根据构造，$\mathrm{D}_{\mathcal{F}}$是一个伪度量，即它是非负对称的（因为$\mathcal{F}=-\mathcal{F}$），当其参数相同时为零，并且满足三角不等式。此外，当$\mathcal{F}$能够区分分布时，$\mathrm{D}_{\mathcal{F}}$成为一个适当的度量，此时$\mathrm{D}_{\mathcal{F}}(\mathbb{P}, \hat{\mathbb{P}})$仅在$\mathbb{P}=\hat{\mathbb{P}}$时为零。关于$\mathrm{D}_{\mathcal{F}}$的模糊集的半径$r \geq 0$，围绕$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$定义为

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{D}_{\mathcal{F}}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}
$$

Proposition 2.11的证明揭示了全变差距离是由所有Borel函数$f: \mathcal{Z} \rightarrow[-1 / 2,1 / 2]$生成的积分概率度量；见(2.16)。在Corollary 2.19中建立的Kantorovich-Rubinstein对偶性进一步表明，1-Wasserstein距离是由所有Lipschitz连续函数$f: \mathcal{Z} \rightarrow \mathbb{R}$且满足$\operatorname{lip}(f) \leq 1$生成的积分概率度量。此外，如果$\mathcal{H}$是一个再生核希尔伯特空间，包含了Borel函数$f: \mathcal{Z} \rightarrow \mathbb{R}$且具有希尔伯特范数$\|\cdot\|_{\mathcal{H}}$，那么与$\mathcal{H}$对应的最大均值差异距离是由标准单位球$\mathcal{F}=\left\{f \in \mathcal{H}:\|f\|_{\mathcal{H}} \leq 1\right\}$在$\mathcal{H}$中生成的积分概率度量。最大均值差异模糊集在(Staib and Jegelka 2019, Zhu, Jitkrittum, Diehl and Schölkopf 2020, 2021, Zeng and Lam 2022, Iyengar, Lam and Wang 2022)中有研究。Husain (2020)揭示了DRO问题与正则化经验风险最小化问题之间的深刻联系，当模糊集通过积分概率度量定义时，此联系成立。


# 3. 模糊集的拓扑性质

理论上和实践中一个重要的问题是，是否自然的子问题（1.2）是可解的，换句话说，是否（1.2）中的内极大值能够被达成。在本节中，我们将探讨在什么条件下，Weierstrass极值定理适用于自然的子问题。也就是说，我们将发展一些容易检查的条件，在这些条件下，模糊集$\mathcal{P}$是弱紧的，并且期望损失$\mathbb{E}_{\mathbb{P}}[\ell(x, Z)]$在$\mathbb{P}$中是弱下半连续的。在整个讨论中，我们假设$\mathcal{Z}$是$\mathbb{R}^d$的一个闭子集。

Baire的经典结果表明，实数线上一个函数是下半连续的当且仅当它可以表示为一系列非递减连续函数的逐点上确界（Baire 1905）。下面我们将使用这一结果的多维推广。

**引理 3.1 (Stromberg (2015, p. 132))**. 一个函数$f: \mathcal{Z} \rightarrow(-\infty,+\infty]$是下半连续的当且仅当存在一个非递减的连续函数序列$f_i: \mathcal{Z} \rightarrow \mathbb{R}, i \in \mathbb{N}$，使得对于所有$z \in \mathcal{Z}$，有$f(z)=\sup _{i \in \mathbb{N}} f_i(z)$。

如果$f$是下界有界的，那么可以假设连续函数$f_i$是均匀有界的。事实上，如果$f(z) \geq 0$，则可以将连续函数$f_i(z)$替换为有界连续函数$f_i^{\prime}(z)=\min \{\max \{f(z), 0\}, i\}$。序列$f_i^{\prime}, i \in \mathbb{N}$仍然是非递减的，并且逐点收敛于$f$。

**定义 3.2 (概率分布的弱收敛)**. 一列概率分布$\mathbb{P}_j \in \mathcal{P}(\mathcal{Z}), j \in \mathbb{N}$，若对于每个有界连续函数$f: \mathcal{Z} \rightarrow \mathbb{R}$，我们有

$$
\lim _{j \in \mathbb{N}} \mathbb{E}_{\mathbb{P}_j}[f(Z)]=\mathbb{E}_{\mathbb{P}}[f(Z)]，
$$

则称$\mathbb{P}_j$弱收敛于$\mathbb{P}$。

期望值$\mathbb{E}_{\mathbb{P}}[f(Z)]$对于分布$\mathbb{P}$的连续性与$f$的连续性特性之间有密切联系。回顾一下，如果一个函数$F: \mathcal{P}(\mathcal{Z}) \rightarrow \overline{\mathbb{R}}$是弱连续的，那么对于每一个收敛到$\mathbb{P}$的概率分布序列$\mathbb{P}_i \in \mathcal{P}(\mathcal{Z}), i \in \mathbb{N}$，都有$\lim _{i \rightarrow \infty} F\left(\mathbb{P}_i\right)=F(\mathbb{P})$。弱下半连续和弱上半连续也可以用类似的方式定义。

**命题 3.3 (期望值的连续性)**. 如果$f: \mathcal{Z} \rightarrow[-\infty,+\infty]$是下半连续且下界有界的，那么$\mathbb{E}_{\mathbb{P}}[f(Z)]$在$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$中是弱下半连续的。反之，如果$f$是上半连续且上界有界的，那么$\mathbb{E}_{\mathbb{P}}[f(Z)]$在$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$中是弱上半连续的。最后，如果$f$是连续且有界的，那么$\mathbb{E}_{\mathbb{P}}[f(Z)]$在$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$中是弱连续的。

证明：假设首先$f$是下半连续且下界有界的。以下我们假设在不失一般性的情况下，$f$实际上是非负的。然后，根据引理3.1，存在一个非递增的有界、连续且非负的函数序列$f_i, i \in \mathbb{N}$，使得$f(z)=\sup _{i \in \mathbb{N}} f_i(z)$。如果$\mathbb{P}_j \in \mathcal{P}(\mathcal{Z}), j \in \mathbb{N}$是任何收敛到$\mathbb{P}$的分布序列，那么我们得到

$$
\begin{aligned}
\liminf _{j \in \mathbb{N}} \mathbb{E}_{\mathbb{P}_j}[f(Z)] & =\sup _{k \in \mathbb{N}} \inf _{j \geq k} \mathbb{E}_{\mathbb{P}_j}\left[\sup _{i \in \mathbb{N}} f_i(Z)\right] \\
& =\sup _{k \in \mathbb{N}} \inf _{j \geq k} \sup _{i \in \mathbb{N}} \mathbb{E}_{\mathbb{P}_j}\left[f_i(Z)\right] \\
& \geq \sup _{i \in \mathbb{N}} \sup _{k \in \mathbb{N}} \inf _{j \geq k} \mathbb{E}_{\mathbb{P}_j}\left[f_i(Z)\right] \\
& =\sup _{i \in \mathbb{N}} \mathbb{E}_{\mathbb{P}}\left[f_i(\xi)\right]=\mathbb{E}_{\mathbb{P}}[f(Z)]
\end{aligned}
$$

这里，第二个等式和最后一个等式都来自于单调收敛定理，该定理适用是因为每个$f_i$是有界的，因此对于任何概率分布都是可积的，并且$f_i, i \in \mathbb{N}$形成一个非递增的非负函数序列。这个不等式是通过交换对$i$的上确界和对$j$的下确界得到的，第三个等式成立是因为$\mathbb{P}_j$收敛到$\mathbb{P}$并且$f_i$是连续且有界的。这表明$\mathbb{E}_{\mathbb{P}}[f(Z)]$在$\mathbb{P}$中是弱下半连续的。

关于弱上半连续性和弱连续性的证明是类似的，因此省略。

接下来，我们为$\mathcal{P}(\mathcal{Z})$上的所有概率分布族配备弱拓扑，这个拓扑是由以下开集生成的：

$$
U_{f, \delta}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}):\left|\mathbb{E}_{\mathbb{P}}[f(Z)]\right|<\delta\right\}
$$

由任何连续有界函数$f: \mathcal{Z} \rightarrow \mathbb{R}$和容忍度$\delta>0$编码。$\mathcal{P}(\mathcal{Z})$上的弱拓扑是由Prokhorov度量度量的（Billingsley 2013, 定理6.8），因此在$\mathcal{P}(\mathcal{Z})$上的序列紧性和紧性是等价的；见例如（Munkres 2000，定理28.2）。

**定义 3.4 (紧性)**. 一个分布族$\mathcal{P} \subseteq \mathcal{P}(\mathcal{Z})$是紧的，当且仅当对于任何容忍度$\varepsilon>0$，存在一个紧集$\mathcal{C} \subseteq \mathcal{Z}$，使得对所有$\mathbb{P} \in \mathcal{P}$有$\mathbb{P}(Z \notin \mathcal{C}) \leq \varepsilon$。

Prokhorov的经典结果表明，分布族是弱紧的当且仅当它是紧的并且弱闭的。Prokhorov定理是证明模糊集是弱紧的关键工具。我们在这里不提供证明。

**定理 3.5 (Billingsley (2013, 定理5.1))**. 一个分布族$\mathcal{P} \subseteq \mathcal{P}(\mathcal{Z})$是弱紧的当且仅当它是紧的并且弱闭的。


## 3.1. 矩模糊集

支持集模糊集无疑是最简单的矩模糊集类，因为它们根本不施加任何矩条件。事实上，本文中考虑的所有其他模糊集都是支持集模糊集的子集。

**命题 3.6 (支持集模糊集)**. 如果$\mathcal{Z} \subseteq \mathbb{R}^d$，则所有支持在$\mathcal{Z}$上的分布的集合$\mathcal{P}(\mathcal{Z})$当且仅当$\mathcal{Z}$是紧的时，弱紧。

**证明**. 首先注意到，$\mathcal{P}(\mathcal{Z})$当且仅当$\mathcal{Z}$是有界的时是紧的。实际上，如果$\mathcal{Z}$是有界的，则它是紧的，因为根据我们的假设$\mathcal{Z}$是闭的。给定任意$\varepsilon>0$，我们可以设置$\mathcal{C}=\mathcal{Z}$，这确保对于所有$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$有$\mathbb{P}(Z \notin \mathcal{C})=0 \leq \varepsilon$。因此，$\mathcal{P}(\mathcal{Z})$是紧的。另一方面，如果$\mathcal{Z}$是无界的，则$\mathcal{P}(\mathcal{Z})$显然不是紧的。实际上，对于任何紧集$\mathcal{C} \subseteq \mathcal{Z}$，补集$\mathcal{Z} \backslash \mathcal{C}$是非空的，因为$\mathcal{C}$是有界的而$\mathcal{Z}$不是。因此，存在一个分布$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$，它的支持在$\mathcal{Z} \backslash \mathcal{C}$上，且$\mathbb{P}(Z \notin \mathcal{C})=1$。

接下来，注意到，$\mathcal{P}(\mathcal{Z})$当且仅当$\mathcal{Z}$是闭的时是弱闭的。首先假设$\mathcal{Z}$是闭的，并且注意到通过定义的指示函数$\delta_{\mathcal{Z}}$，当$z \in \mathcal{Z}$时$\delta_{\mathcal{Z}}(z)=0$，否则$\delta_{\mathcal{Z}}(z)=+\infty$，是下半连续且下界有界的。根据命题3.3，$\mathbb{E}_{\mathbb{P}}\left[\delta_{\mathcal{Z}}(Z)\right]$因此在$\mathbb{P}$中是弱下半连续的。如果$\mathbb{P}_j \in \mathcal{P}(\mathcal{Z}), j \in \mathbb{N}$，收敛到$\mathbb{P}$，那么我们有

$$
0=\liminf _{j \in \mathbb{N}} \mathbb{E}_{\mathbb{P}_j}\left[\delta_{\mathcal{Z}}(Z)\right] \geq \mathbb{E}_{\mathbb{P}}\left[\delta_{\mathcal{Z}}(Z)\right] \geq 0
$$

其中，等式成立是因为对于每个$j \in \mathbb{N}$，$\mathbb{P}_j$支持在$\mathcal{Z}$上，第一个不等式来自弱下半连续性。这表明$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$，因此$\mathcal{P}(\mathcal{Z})$是弱闭的。反之，假设$\mathcal{P}(\mathcal{Z})$是弱闭的，考虑一个序列$z_j \in \mathcal{Z}, j \in \mathbb{N}$，收敛到$z$。然后，狄拉克分布序列$\delta_{z_j}, j \in \mathbb{N}$收敛到$\delta_z$，因此我们得到

$$
0=\liminf _{j \in \mathbb{N}} \mathbb{E}_{\delta_{z j}}\left[\delta_{\mathcal{Z}}(Z)\right] \geq \mathbb{E}_{\delta_z}\left[\delta_{\mathcal{Z}}(Z)\right] \geq 0
$$

这里，第一个不等式再次成立，因为$\mathbb{E}_{\mathbb{P}}\left[\delta_{\mathcal{Z}}(Z)\right]$在$\mathbb{P}$中是弱下半连续的。这表明$\mathbb{E}_{\delta_z}\left[\delta_{\mathcal{Z}}(Z)\right]=0$，当且仅当$z \in \mathcal{Z}$时成立。因此，$\mathcal{Z}$是闭的。根据这些见解，结论可以通过定理3.5得出。
通过使用命题3.6，我们现在可以证明，如果基础支撑集 $\mathcal{Z}$ 是紧的，矩量函数 $f$ 是连续的，并且不确定集 $\mathcal{F}$ 是闭的，那么形式为(2.1)的矩量模糊集是弱紧的。

**命题3.7（矩量模糊集）**  
如果 $\mathcal{Z} \subseteq \mathbb{R}^d$ 是紧的支撑集，$f: \mathcal{Z} \rightarrow \mathbb{R}^m$ 是连续的矩量函数，且 $\mathcal{F} \subseteq \mathbb{R}^m$ 是闭的不确定集，则按(2.1)定义的矩量模糊集 $\mathcal{P}$ 是弱紧的。

**证明：**  
由于 $\mathcal{Z}$ 是紧的，仅支撑集的模糊集 $\mathcal{P}(\mathcal{Z})$ 根据命题3.6是弱紧的。因此，$\mathcal{P}(\mathcal{Z})$ 是紧的并且弱闭的。这很容易推导出，$\mathcal{P}$ 作为紧集的子集仍然是紧的。命题3.3进一步表明，$\mathbb{E}_{\mathbb{P}}[f(Z)]$ 在 $\mathbb{P}$ 上是弱连续的。由于 $\mathcal{F}$ 是闭的，并且任何闭集在连续变换下的原像也是闭的，我们可以得出结论，$\mathcal{P}_f=\left\{\mathbb{P} \in \mathcal{P}(\mathbb{R}^d): \mathbb{E}_{\mathbb{P}}[f(Z)] \in \mathcal{F}\right\}$ 是弱闭的。因此，$\mathcal{P}=\mathcal{P}(\mathcal{Z}) \cap \mathcal{P}_f$ 是弱闭的，因为它是两个弱闭集的交集。根据这些见解，结论从定理3.5中可以直接推导出来。

命题3.7的条件仅是弱紧的充分条件，而非必要条件。以下示例表明，即使支撑集 $\mathcal{Z}$ 或矩量函数 $f$ 是无界的，矩量模糊集仍然可以是紧的或弱紧的。

**例3.8（马尔科夫模糊集）**  
如果 $\mathcal{Z}=\mathbb{R}^d$，则马尔科夫模糊集(2.2)可能不紧。例如，如果 $\mathcal{Z}=\mathbb{R}$ 且 $\mu=0$，那么对于每个紧集 $\mathcal{C} \subseteq \mathbb{R}$，都有一个常数 $R>0$，使得两点分布 $\mathbb{P}=\frac{1}{2} \delta_{-R}+\frac{1}{2} \delta_R$ 完全支持于 $\mathcal{C}$ 的补集上。然而，如果 $\mathcal{Z}=\mathbb{R}_{+}$ 且 $\mu=1$，则马尔科夫不等式意味着，对于每个 $\mathbb{P} \in \mathcal{P}$ 和任意 $\varepsilon>0$，都可以定义紧集 $\mathcal{C}$ 为区间 $[0, 1/\varepsilon]$，使得 $\mathbb{P}(Z \notin \mathcal{C}) \leq \varepsilon$。即使在这种情况下，$\mathcal{P}$ 也不是弱闭的。实际上，分布 $\mathbb{P}_i=\frac{i}{i+1} \delta_0+\frac{1}{i+1} \delta_{i+1}$ 对于所有 $i \in \mathbb{N}$ 都属于 $\mathcal{P}$，但它们的弱极限 $\mathbb{P}=\delta_0$ 并不属于 $\mathcal{P}$。如果 $\mathcal{Z}$ 是凸的，则可以扩展这一推理，证明只有当 $\mathcal{Z}$ 是紧的时，$\mathcal{P}$ 才是弱紧的。

**例3.9（切比雪夫模糊集）**  
切比雪夫模糊集 $\mathcal{P}$ 定义在(2.3)中，总是紧的。为了说明这一点，假设 $\mu=0$ 且 $M=I_d$，通过仿射坐标变换，可以始终满足这一条件。给定任意的 $\varepsilon>0$，我们可以定义紧集 $\mathcal{C}=\{z \in \mathcal{Z} : \|z\|_2 \leq \sqrt{d/\varepsilon}\}$。然后很容易看出，任何 $\mathbb{P} \in \mathcal{P}$ 都满足

$$
\mathbb{P}(Z \notin \mathcal{C})=\mathbb{P}(\|Z\|_2>\sqrt{d/\varepsilon}) \leq \mathbb{E}_{\mathbb{P}}[\|Z\|_2^2 \cdot \varepsilon / d] = \varepsilon,
$$

其中不等式成立是因为二次函数 $q(z)=\|z\|_2^2 \cdot \varepsilon / d$ 优于 $\mathcal{Z} \backslash \mathcal{C}$ 的特征函数。因此，$\mathcal{P}$ 确实是紧的。然而，$\mathcal{P}$ 不一定是弱闭的。为了看到这一点，假设 $d=1$ 且 $\mathcal{Z}=\mathbb{R}$。在这种情况下，分布 $\mathbb{P}_i=\frac{1}{2i^2} \delta_{-i} + \frac{i^2-1}{i^2} \delta_0 + \frac{1}{2i^2} \delta_i$ 对于所有 $i \in \mathbb{N}$ 都有零均值和单位方差，且它们都属于 $\mathcal{P}$。然而，它们收敛于弱极限 $\mathbb{P}=\delta_0$，后者不属于 $\mathcal{P}$。因此，$\mathcal{P}$ 不是弱紧的。

所有具有有限 $p$-阶矩的 $\mathbb{R}^d$ 上的分布族总是弱紧的，即使那些固定 $p$-阶矩的模糊集（例如切比雪夫模糊集）可能不是弱紧的。

**例3.10（$p$-阶矩模糊集）**  
由任何范数 $\|\cdot\|$ 在 $\mathbb{R}^d$ 上诱导的模糊集

$$
\mathcal{P} = \left\{ \mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_p[\|Z\|^p] \leq R \right\}
$$

对于任意的 $p, R > 0$ 总是弱紧的。使用类似于例3.9中的推理，可以证明，对于任意的 $\varepsilon>0$，存在一个紧集，定义为 $\mathcal{C} = \left\{ z \in \mathcal{Z} : \|z\| \leq (R/\varepsilon)^{1/p} \right\}$，满足 $\mathbb{P}(Z \notin \mathcal{C}) \leq \varepsilon$。因此，$\mathcal{P}$ 是紧的。为了证明 $\mathcal{P}$ 也是弱闭的，注意到 $f(z)=\|z\|^p$ 是连续的并且下界有界。根据命题3.3，期望值 $\mathbb{E}_{\mathbb{P}}[\|Z\|_2^p]$ 在 $\mathbb{P}$ 上是弱下半连续的，且其子水平集是弱闭的。因此，$\mathcal{P}$ 是弱紧的，由定理3.5得出。
